In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance



import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions
import src.analysis.utils.general_utils as utils # import utils functions one by one by name

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


choose which subjects you wanna run (has to be a list, even if just one subject)

In [28]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

### make or load subjects electrodes to rois dict

In [29]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [30]:
subjects_electrodestoROIs_dict

{'D0057': {'default_dict': {'RAI1': 'ctx_rh_G_Ins_lg_and_S_cent_ins',
   'RAI2': 'Right-Cerebral-White-Matter',
   'RAI3': 'Right-Cerebral-White-Matter',
   'RAI4': 'Right-Cerebral-White-Matter',
   'RAI5': 'Right-Cerebral-White-Matter',
   'RAI6': 'ctx_rh_S_circular_insula_sup',
   'RAI7': 'Right-Cerebral-White-Matter',
   'RAI8': 'Right-Cerebral-White-Matter',
   'RAI9': 'Right-Cerebral-White-Matter',
   'RAI10': 'Right-Cerebral-White-Matter',
   'RAI11': 'Right-Cerebral-White-Matter',
   'RAI12': 'ctx_rh_S_front_sup',
   'RAI13': 'ctx_rh_S_front_sup',
   'RAI14': 'ctx_rh_S_front_sup',
   'RAI15': 'ctx_rh_G_front_sup',
   'RAI16': 'ctx_rh_G_front_sup',
   'RPI1': 'Right-Cerebral-White-Matter',
   'RPI2': 'Right-Cerebral-White-Matter',
   'RPI3': 'Right-Cerebral-White-Matter',
   'RPI4': 'Right-Cerebral-White-Matter',
   'RPI5': 'Right-Cerebral-White-Matter',
   'RPI6': 'Right-Cerebral-White-Matter',
   'RPI7': 'Right-Cerebral-White-Matter',
   'RPI8': 'Right-Cerebral-White-Matter',
 

In [31]:
def count_electrodes_across_subjects(data, subjects):
    total_electrodes = 0
    for subject, details in data.items():
        if subject in subjects:
            total_electrodes += len(details['default_dict'])
    return total_electrodes

# Calculate the total number of electrodes in the 'default_dict' across subjects
total_electrodes = count_electrodes_across_subjects(subjects_electrodestoROIs_dict, subjects)
print('total elecs:', total_electrodes)

total elecs: 3277


load behavioral data (combinedData.csv)

In [32]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

map blockType to congruency and switch proportions in the behavioral data

In [33]:
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(utils.map_block_type, axis=1)

### load evoked and stuff for all subjects in a dictionary

In [40]:
task='GlobalLocal'
conditions = experiment_conditions.stimulus_switch_type_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = [condition for condition in conditions.keys()]

In [595]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    
# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline corr

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:740: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  Structure: `data_timeAvg_lists[time_window_suffix][condition_name][roi_name] = list_of_time_avg_arrays`.


  Loading condition: Stimulus_s with parameters: {'BIDS_events': ['Stimulus/s25.0', 'Stimulus/s75.0'], 'switchType': 's'}
Not setting metadata
189 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:740: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  Structure: `data_timeAvg_lists[time_window_suffix][condition_name][roi_name] = list_of_time_avg_arrays`.


  Loading condition: Stimulus_r with parameters: {'BIDS_events': ['Stimulus/r25.0', 'Stimulus/r75.0'], 'switchType': 'r'}
Not setting metadata
212 matching events found
No baseline correction applied


...

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:740: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  Structure: `data_timeAvg_lists[time_window_suffix][condition_name][roi_name] = list_of_time_avg_arrays`.


No baseline correction applied
  Loading condition: Stimulus_r with parameters: {'BIDS_events': ['Stimulus/r25.0', 'Stimulus/r75.0'], 'switchType': 'r'}
Not setting metadata
204 matching events found


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:740: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  Structure: `data_timeAvg_lists[time_window_suffix][condition_name][roi_name] = list_of_time_avg_arrays`.


No baseline correction applied
  Loading condition: Stimulus_s with parameters: {'BIDS_events': ['Stimulus/s25.0', 'Stimulus/s75.0'], 'switchType': 's'}
Not setting metadata
193 matching events found
No baseline correction applied


c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:740: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  Structure: `data_timeAvg_lists[time_window_suffix][condition_name][roi_name] = list_of_time_avg_arrays`.


### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [41]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103
Loaded significant channels for subject D0107A
Loaded significant channels for subject D0110
Loaded significant channels for subject D0116
Loaded significant channels for subject D0117
Loaded significant channels for subject D0121


In [42]:
sig_chans_per_subject

{'D0057': ['RAI1',
  'RAI2',
  'RAI3',
  'RAI4',
  'RAI5',
  'RAI6',
  'RAI8',
  'RAI9',
  'RAI10',
  'RAI12',
  'RAI13',
  'RAI14',
  'RPI1',
  'RPI2',
  'RPI3',
  'RPI4',
  'RPI5',
  'RPI6',
  'RPI7',
  'RPI8',
  'RPI10',
  'RPI11',
  'RAIF1',
  'RAIF2',
  'RAIF3',
  'RAIF4',
  'RAIF5',
  'RAIF6',
  'RAIF7',
  'RAIF8',
  'RAIF9',
  'RAIF11',
  'RAIF12',
  'RAIF13',
  'RAIF14',
  'RAMF1',
  'RAMF2',
  'RAMF3',
  'RAMF4',
  'RAMF5',
  'RAMF6',
  'RAMF7',
  'RAMF8',
  'RAMF10',
  'RAMF11',
  'RAMF12',
  'RAMF14',
  'RMMF1',
  'RMMF2',
  'RMMF3',
  'RMMF4',
  'RMMF5',
  'RMMF6',
  'RMMF8',
  'RMMF9',
  'RMMF10',
  'RMMF11',
  'RMMF12',
  'RMMF13',
  'RPIP1',
  'RPIP2',
  'RPIP3',
  'RPIP4',
  'RPIP5',
  'RPIP6',
  'RPIP7',
  'RPIP8',
  'RPIP9',
  'RPIP11',
  'RPIP13',
  'RAST1',
  'RAST2',
  'RAST3',
  'RAST4',
  'RAST7',
  'RAIT1',
  'RAIT2',
  'RAIT3',
  'RAIT4',
  'RAIT5',
  'RAIT6',
  'RAIT7',
  'RAIT8',
  'RAIT9',
  'RAMT1',
  'RAMT2',
  'RAMT3',
  'RAMT4',
  'RAMT6',
  'RAMT7',
  '

### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [43]:
# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF

In [44]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 1309


get total number of electrodes (make this modular with roi later once everything works)

In [45]:
# Example usage:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 107
Total number of lpfc electrodes across all subjects: 267
Total number of significant occ electrodes across all subjects: 34
Total number of occ electrodes across all subjects: 76


check if any subjects have a different sampling rate

In [601]:
def check_sampling_rates(subjects_mne_objects, expected_sampling_rate=256):
    # This dictionary will store subjects with their sampling rates
    subject_sampling_rates = {}

    # Iterate through each subject and their corresponding data
    for subject, data in subjects_mne_objects.items():
        # Get the first epochs object from the dictionary
        if data:
            first_condition = list(data.keys())[0]
            mne_objects = data[first_condition]
            first_object = list(mne_objects.keys())[0]
            first_epochs = data[first_condition][first_object]
            actual_sampling_rate = first_epochs.info['sfreq']
            
            # Store the sampling rate in the dictionary
            subject_sampling_rates[subject] = actual_sampling_rate
    
    # Print the results
    for subject, rate in subject_sampling_rates.items():
        if rate != expected_sampling_rate:
            print(f"Subject {subject} has a different sampling rate: {rate} Hz.")
        else:
            print(f"Subject {subject} has the expected sampling rate: {rate} Hz.")
    
    return subject_sampling_rates

# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.
Subject D0107A has the expected sampling rate: 256.0 Hz.
Subject D0110 has the expected sampling rate: 256.0 Hz.
Subject D0116 has the expected sampling rate: 256.0 Hz.
Subject D0117 has the expected sampling rate: 256.0 Hz.
Subject D0121 has the expected sampling rate: 256.0 Hz.


### do stats

current approach is to run time_perm_cluster on significant dlpfc electrodes for each subject, comparing congruent and incongruent conditions. Then, average p-values across all subjects. Discuss this with Greg, probably wrong approach.

**1/23 new approach is to average across all trials for sig dlpfc electrodes, comparing incongruent and congruent conditions. Then, run stats on this new avg electrode value x time array.

Also, I'm using HG_ev1_rescaled instead of HG_ev1 to compare congruent and incongruent, so that they're normalized with a common baseline. I think this is better than comparing the raw HG traces directly.

### this is 1/23 old approach of avg across trials first. Time perm cluster stats.

do stats and plotting together. Stats needs trial avg data, plotting just needs congruent_data without trial averaging (initially at least)  
this code is so bad right now, turn into a function later  

trialAvg is for the time perm cluster stats  
timeAvg_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond is for the window stats (not sure if this is even right)  



4/30 try to make time perm stats more modular, and reusable  
also remember that time perm cluster stats only compares two output names.  

these functions are now in utils.py. 5/6.

In [602]:
# # Define time indices as in your original code
# time_indices = {
#     'firstHalfSecond': (2048, 3072),
#     'secondHalfSecond': (3072, 4096),
#     'fullSecond': (2048, 4096)
# }

# # Select output names based on whether the processing is for a permutation test (first two outputs) or ANOVA (all outputs).
# for_perm_test = True
# relevant_condition_names = condition_names[:2] if for_perm_test else condition_names

# # Process the data
# data_trialAvg_lists, data_trialStd_lists, data_timeAvg_lists, overall_electrode_mapping, electrode_mapping_per_roi = utils.process_data_for_roi(
#     subjects_mne_objects, relevant_condition_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

# # Concatenate the data
# concatenated_trialAvg_data = utils.concatenate_data(data_trialAvg_lists, rois, relevant_condition_names)

# # Calculate means and sems
# mean_and_sem = utils.calculate_mean_and_sem(concatenated_trialAvg_data, rois, relevant_condition_names)

# # Perform statistical tests
# time_perm_cluster_results = utils.calculate_time_perm_cluster_for_each_roi(concatenated_trialAvg_data, rois, relevant_condition_names)

### do 2x2 anova for interaction effects 
this requires reloading in all four conditions (four this time cuz interaction contrasts).  
ONLY RUN THIS WHEN LOADING IN THE FOUR INTERACTION CONTRASTS RIGHT NOW.  
Integrate with other stats and plotting and stuff later.

In [603]:
# i should turn this all into a function too. 5/20.
# define time windows in terms of samples (this is cuz epochs are from -1 to 1.5 sec after stimulus onset)
# time_indices = {
#     'firstHalfSecond': (2048, 3072),
#     'secondHalfSecond': (3072, 4096),
#     'fullSecond': (2048, 4096)
# }

original_time_indices = {
    'firstHalfSecond': (2048, 3072),
    'secondHalfSecond': (3072, 4096)
}

sampling_rate_ratio = sampling_rate / 2048.0 # account for decimation factor

# Convert the time indices to the new sampling rate
time_indices = {
    key: (int(start * sampling_rate_ratio), int(end * sampling_rate_ratio))
    for key, (start, end) in original_time_indices.items()
}

condition_names = list(conditions.keys()) # get the condition names as a list

# # Select output names based on whether the processing is for a permutation test (first two outputs) or ANOVA (all outputs).
# for_perm_test = False
# relevant_condition_names = condition_names[:2] if for_perm_test else condition_names

# Process the data (use sig_electrodes_per_subject_roi if just using sig elecs, and use electrodes_per_subject_roi if using all elecs)
data_trialAvg_lists, data_trialStd_lists, data_timeAvg_lists, overall_electrode_mapping, electrode_mapping_per_roi = utils.process_data_for_roi(
    subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

print("Data processing complete. Now let's concatenate the results and see what we've got!")

# Concatenate the data
concatenated_trialAvg_data = utils.concatenate_data(data_trialAvg_lists, rois, condition_names)

# Calculate means and sems
mean_and_sem = utils.calculate_mean_and_sem(concatenated_trialAvg_data, rois, condition_names)

print("Mean and SEM calculation done")

Subject: D0057, ROI: lpfc, Num of Sig Electrodes: 2
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0057, ROI: occ, Num of Sig Electrodes: 1
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0059, ROI: lpfc, Num of Sig Electrodes: 3
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0059, ROI: occ, Num of Sig Electrodes: 0
Subject: D0063, ROI: lpfc, Num of Sig Electrodes: 7
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0063, ROI: occ, Num of Sig Electrodes: 0
Subject: D0065, ROI: lpfc, Num of Sig Electrodes: 0
Subject: D0065, ROI: occ, Num of Sig Electrodes: 0
Subject: D0069, ROI: lpfc, Num of Sig Electrodes: 0
Subject: D0069, ROI: occ, Nu

make dataframe for anova

In [604]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Example structure for organizing data for ANOVA with four conditions

# Function to process and append data for ANOVA from time-averaged lists
# Adapted function to include Congruency and SwitchType
# modifying this to include time windows as another factor 4/4! Use code before 4/4 if don't want to include time windows.
def process_and_append_data_for_anova(time_averaged_lists_dict, conditions):
    """
    Transforms nested time-averaged electrophysiological data into a flat list
    of dictionaries, suitable for creating a Pandas DataFrame for ANOVA.

    The function iterates through time windows, experimental conditions,
    Regions of Interest (ROIs), subjects, and significant electrodes. For each
    electrode of each subject, it calculates the mean activity (by averaging
    over the first dimension of the subject-specific data, typically trials)
    and compiles this along with subject, electrode, ROI, time window, and
    condition-specific parameters into a dictionary.

    Parameters:
    ----------
    time_averaged_lists_dict : dict
        A nested dictionary containing the time-averaged data.
        Expected structure:
        `time_averaged_lists_dict[time_window_name][condition_name][roi_name]`
        This path should lead to a list of NumPy arrays. Each array in this list
        represents data for one subject and is typically shaped
        `(n_trials, n_significant_electrodes_for_this_subject_in_this_roi)`.
        Example: `{'FirstHalfSecond': {'ConditionA': {'dlPFC': [subj1_data, subj2_data, ...]}}}`
    conditions : dict
        A dictionary defining the experimental conditions and their parameters.
        Keys are condition names (e.g., 'Stimulus_cr') matching those in
        `time_averaged_lists_dict`. Values are dictionaries containing the
        parameter names and their specific values for that condition, which will
        be added as columns in the output.
        Example: `{'Stimulus_cr': {'Congruency': 'Congruent', 'SwitchType': 'Repeat'}}`

    Returns:
    -------
    list of dict
        A list where each dictionary represents a single row for an ANOVA DataFrame.
        Each dictionary contains the following keys:
        - 'SubjectID': Subject identifier.
        - 'Electrode': Electrode name.
        - 'ROI': Region of Interest.
        - 'TimeWindow': The time window from which the data was processed.
        - 'MeanActivity': The mean activity for that electrode, subject, ROI,
                          time window, and condition (averaged across trials).
        - Keys from `condition_parameters`: Additional columns based on the
          experimental design (e.g., 'Congruency', 'SwitchType').

    Notes:
    -----
    - This function assumes that `rois` (a list of ROI names) and
      `sig_electrodes_per_subject_roi` (a dict mapping
      `[roi_name][subject_id]` to a list of significant electrode names)
      are available in the scope where the function is called.
    - The number of items in the list `lists[condition][roi]` (i.e., number of subjects
      with data for that condition/ROI) must correspond to the order of subjects
      derived from `subjects_with_data` for correct SubjectID mapping.
    """    
    data_for_anova = []
    for time_window, lists in time_averaged_lists_dict.items():
        for condition, condition_parameters in conditions.items():
            print('condition:', condition)
            # # Dynamically get condition parameters and their values for the current condition_name
            # condition_parameters = conditions[condition]
            
            for roi in rois: #this is good cuz it loops through rois 3/6, the trial level one should copy this logic
                sig_electrodes_per_subject = sig_electrodes_per_subject_roi[roi]
                subjects_with_data = [subject for subject, electrodes in sig_electrodes_per_subject.items() if electrodes] # add this line to skip over subjects without data 4/1
                for subject_index, subject_data in enumerate(lists[condition][roi]):
                    subject_id = subjects_with_data[subject_index]

                    # Skip this subject if there are no significant electrodes for them in this ROI
                    if subject_id not in sig_electrodes_per_subject or not sig_electrodes_per_subject[subject_id]:
                        continue

                    # Calculate the mean across trials for each electrode
                    mean_activity_per_electrode = np.nanmean(subject_data, axis=0)
                    # untested making this more modular 2/27
                    for electrode_index, mean_activity in enumerate(mean_activity_per_electrode):
                        print('electrode index:', electrode_index)
                        electrode_name = sig_electrodes_per_subject[subject_id][electrode_index]
                        print(electrode_name)
                        # Prepare data dictionary, starting with fixed attributes
                        data_dict = {
                            'SubjectID': subject_id,
                            'Electrode': electrode_name,
                            'ROI': roi,
                            'TimeWindow': time_window,
                            'MeanActivity': mean_activity
                        }

                        # Dynamically add condition types and their values
                        data_dict.update(condition_parameters)

                        # Append the organized data to the list
                        data_for_anova.append(data_dict)
    return data_for_anova

# Create a time averaged lists dictionary to pass in to the process and append data for anova function
                    
# use this one to compare early vs late vs all time
# time_averaged_lists = {
#         "FirstHalfSecond": output_data_timeAvg_firstHalfSecond_lists,
#         "SecondHalfSecond": output_data_timeAvg_secondHalfSecond_lists,
#         "FullSecond": output_data_timeAvg_fullSecond_lists
# }
                    
# # use this one to just compare early and late time
# time_averaged_lists_dict = {
#         "FirstHalfSecond": output_data_timeAvg_firstHalfSecond_lists,
#         "SecondHalfSecond": output_data_timeAvg_secondHalfSecond_lists
# }
#

data_for_anova = process_and_append_data_for_anova(data_timeAvg_lists, conditions)
# Convert to DataFrame
df_for_anova = pd.DataFrame(data_for_anova)

condition: Stimulus_r
electrode index: 0
RAI6
electrode index: 1
RAMF14
electrode index: 0
LMMF11
electrode index: 1
LMMF10
electrode index: 2
LPSF16
electrode index: 0
LPSF12
electrode index: 1
RMMF13
electrode index: 2
RMMF14
electrode index: 3
RAI4
electrode index: 4
RAI6
electrode index: 5
RAI5
electrode index: 6
RMMF10
electrode index: 0
RIA4
electrode index: 1
RIP6
electrode index: 2
RIA5
electrode index: 0
RIA6
electrode index: 1
RIA11
electrode index: 0
LFAM8
electrode index: 1
LFAM9
electrode index: 2
LFAM10
electrode index: 3
LFPM10
electrode index: 4
LFPM12
electrode index: 5
LFAI9
electrode index: 6
LFAI10
electrode index: 7
LIA11
electrode index: 0
RFO13
electrode index: 1
RFO14
electrode index: 0
LFAM8
electrode index: 1
LFAM9
electrode index: 2
LAI13
electrode index: 3
LAI14
electrode index: 4
LAI4
electrode index: 5
LAI7
electrode index: 6
LAI8
electrode index: 7
LFAI4
electrode index: 0
RFAI3
electrode index: 0
LFOA18
electrode index: 1
LFMI3
electrode index: 2
RFMM12


In [605]:
df_for_anova

,SubjectID,Electrode,ROI,TimeWindow,MeanActivity,BIDS_events,switchType
0,D0057,RAI6,lpfc,firstHalfSecond,0.266503,"[Stimulus/r25.0, Stimulus/r75.0]",r
1,D0057,RAMF14,lpfc,firstHalfSecond,0.331035,"[Stimulus/r25.0, Stimulus/r75.0]",r
2,D0059,LMMF11,lpfc,firstHalfSecond,0.060583,"[Stimulus/r25.0, Stimulus/r75.0]",r
3,D0059,LMMF10,lpfc,firstHalfSecond,-0.031626,"[Stimulus/r25.0, Stimulus/r75.0]",r
4,D0059,LPSF16,lpfc,firstHalfSecond,0.103240,"[Stimulus/r25.0, Stimulus/r75.0]",r
...,...,...,...,...,...,...,...
247,D0100,LOPM8,occ,secondHalfSecond,0.691395,"[Stimulus/s25.0, Stimulus/s75.0]",s
248,D0100,LOPM9,occ,secondHalfSecond,0.436370,"[Stimulus/s25.0, Stimulus/s75.0]",s
249,D0103,LTPI2,occ,secondHalfSecond,0.220610,"[Stimulus/s25.0, Stimulus/s75.0]",s
250,D0103,LTPI3,occ,secondHalfSecond,0.037067,"[Stimulus/s25.0, Stimulus/s75.0]",s


In [606]:
sig_electrodes_per_subject_roi

{'lpfc': {'D0057': ['RAI6', 'RAMF14'],
  'D0059': ['LMMF11', 'LMMF10', 'LPSF16'],
  'D0063': ['LPSF12', 'RMMF13', 'RMMF14', 'RAI4', 'RAI6', 'RAI5', 'RMMF10'],
  'D0065': [],
  'D0069': [],
  'D0071': ['RIA4', 'RIP6', 'RIA5'],
  'D0077': [],
  'D0090': ['RIA6', 'RIA11'],
  'D0094': ['LFAM8',
   'LFAM9',
   'LFAM10',
   'LFPM10',
   'LFPM12',
   'LFAI9',
   'LFAI10',
   'LIA11'],
  'D0100': [],
  'D0102': ['RFO13', 'RFO14'],
  'D0103': ['LFAM8',
   'LFAM9',
   'LAI13',
   'LAI14',
   'LAI4',
   'LAI7',
   'LAI8',
   'LFAI4'],
  'D0107A': ['RFAI3'],
  'D0110': [],
  'D0116': ['LFOA18', 'LFMI3', 'RFMM12', 'RAI15', 'RAI17'],
  'D0117': ['RINSA3'],
  'D0121': ['LFPS2', 'LFA11']},
 'occ': {'D0057': ['RPIT1'],
  'D0059': [],
  'D0063': [],
  'D0065': [],
  'D0069': [],
  'D0071': [],
  'D0077': ['ROPM11', 'ROPM1'],
  'D0090': ['RTPI2'],
  'D0094': [],
  'D0100': ['LTOJ1',
   'LTPI1',
   'LOPM1',
   'LOMM3',
   'LTOJ13',
   'LOMI11',
   'LOPI8',
   'LOPI9',
   'LOPM10',
   'LOPS12',
   'LOPM8',

now actually run anova

In [607]:
df_for_anova

,SubjectID,Electrode,ROI,TimeWindow,MeanActivity,BIDS_events,switchType
0,D0057,RAI6,lpfc,firstHalfSecond,0.266503,"[Stimulus/r25.0, Stimulus/r75.0]",r
1,D0057,RAMF14,lpfc,firstHalfSecond,0.331035,"[Stimulus/r25.0, Stimulus/r75.0]",r
2,D0059,LMMF11,lpfc,firstHalfSecond,0.060583,"[Stimulus/r25.0, Stimulus/r75.0]",r
3,D0059,LMMF10,lpfc,firstHalfSecond,-0.031626,"[Stimulus/r25.0, Stimulus/r75.0]",r
4,D0059,LPSF16,lpfc,firstHalfSecond,0.103240,"[Stimulus/r25.0, Stimulus/r75.0]",r
...,...,...,...,...,...,...,...
247,D0100,LOPM8,occ,secondHalfSecond,0.691395,"[Stimulus/s25.0, Stimulus/s75.0]",s
248,D0100,LOPM9,occ,secondHalfSecond,0.436370,"[Stimulus/s25.0, Stimulus/s75.0]",s
249,D0103,LTPI2,occ,secondHalfSecond,0.220610,"[Stimulus/s25.0, Stimulus/s75.0]",s
250,D0103,LTPI3,occ,secondHalfSecond,0.037067,"[Stimulus/s25.0, Stimulus/s75.0]",s


In [608]:
rois

['lpfc', 'occ']

In [609]:
def perform_modular_anova_all_time_windows_per_roi(df, conditions, rois, save_dir, save_name_prefix):
    """
    Performs an ANOVA test separately for each ROI on the provided DataFrame,
    including 'TimeWindow' as a factor, and saves the results to text files.

    The ANOVA model formula is dynamically constructed based on the keys
    found in the `conditions` dictionary and explicitly includes
    'TimeWindow' as a categorical variable. It models main effects for all
    conditions and 'TimeWindow', as well as all their interaction terms.
    The dependent variable is assumed to be 'MeanActivity'.

    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing the data for ANOVA. Must include
        a 'TimeWindow' column (which will be treated as a categorical factor),
        an 'ROI' column, and a 'MeanActivity' column as the dependent variable.
        It should also contain columns corresponding to the condition keys derived
        from `conditions`.
    conditions : dict
        A dictionary defining the experimental conditions. The keys of
        the inner dictionary (for any given condition name) are used to
        construct the formula terms related to experimental conditions.
        Example: `{'Stimulus_cr': {'congruency': 'c', 'switchType': 'r'}}`
    rois : list
        A list of ROI names to iterate over. ANOVA will be performed separately for each.
    save_dir : str
        The absolute path to the directory where the ANOVA results files
        will be saved.
    save_name_prefix : str
        A prefix for the output filenames. The final filename will be
        `{save_name_prefix}_{roi_name}_ANOVAacrossElectrodes_allTimeWindows.txt`.

    Returns:
    -------
    dict
        A dictionary where keys are ROI names and values are the
        ANOVA results table object (statsmodels.iolib.table.SimpleTable or anova_lm object)
        from `statsmodels.stats.anova.anova_lm` for that ROI.

    Notes:
    -----
    - The function uses a Type II ANOVA (`typ=2`).
    - The formula includes main effects for each condition key, 'TimeWindow',
      and all possible interactions between them.
      Example formula structure:
      `MeanActivity ~ C(key1) + ... + C(TimeWindow) + C(key1)*...*C(TimeWindow)`
    - Ensure that `df` contains all necessary columns specified by the
      dynamically generated formula and the 'MeanActivity', 'TimeWindow', and 'ROI' columns.
      'TimeWindow' should contain discrete categories.
    """
    all_anova_results = {}

    for roi in rois:
        print(f"Performing ANOVA for ROI: {roi}")
        df_roi = df[df['ROI'] == roi]

        if df_roi.empty:
            print(f"No data found for ROI: {roi}. Skipping ANOVA.")
            continue

        # Dynamically construct the model formula based on condition keys and include TimeWindow and roi 5/20. Exclude BIDS_events.
        condition_keys = [key for key in conditions[next(iter(conditions))].keys() if key!= 'BIDS_events']
        formula_terms = ' + '.join([f'C({key})' for key in condition_keys] + ['C(TimeWindow)'])
        interaction_terms = ' * '.join([f'C({key})' for key in condition_keys] + ['C(TimeWindow)'])
        formula = f'MeanActivity ~ {formula_terms} + {interaction_terms}'

        # Define the model
        model = ols(formula, data=df_roi).fit()

        # Perform the ANOVA
        anova_results = anova_lm(model, typ=2)
        all_anova_results[roi] = anova_results

        # Define the base part of the results file name
        results_file_path = os.path.join(save_dir, f"{save_name_prefix}_{roi}_ANOVAacrossElectrodes_allTimeWindows.txt")

        # Save the ANOVA results to a text file
        with open(results_file_path, 'w') as file:
            file.write(anova_results.__str__())

        # Optionally, print the path to the saved file and/or return it
        print(f"ANOVA results for all time windows saved to: {results_file_path}")

        # Print the results
        print(anova_results)

    return all_anova_results

run anova

In [610]:
perform_modular_anova_all_time_windows_per_roi(df_for_anova, conditions, rois, save_dir, f'{conditions_save_name}')

Performing ANOVA for ROI: lpfc
ANOVA results for all time windows saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\stimulus_switch_type_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_17_subjects_lpfc_ANOVAacrossElectrodes_allTimeWindows.txt
                               sum_sq     df          F    PR(>F)
C(switchType)                0.016917    1.0   0.307936  0.579671
C(TimeWindow)                0.734708    1.0  13.373737  0.000339
C(switchType):C(TimeWindow)  0.037339    1.0   0.679670  0.410842
Residual                     9.449094  172.0        NaN       NaN
Performing ANOVA for ROI: occ
ANOVA results for all time windows saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\stimulus_switch_type_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_7

{'lpfc':                                sum_sq     df          F    PR(>F)
 C(switchType)                0.016917    1.0   0.307936  0.579671
 C(TimeWindow)                0.734708    1.0  13.373737  0.000339
 C(switchType):C(TimeWindow)  0.037339    1.0   0.679670  0.410842
 Residual                     9.449094  172.0        NaN       NaN,
 'occ':                                sum_sq    df         F    PR(>F)
 C(switchType)                0.000043   1.0  0.000320  0.985781
 C(TimeWindow)                0.460394   1.0  3.415248  0.068704
 C(switchType):C(TimeWindow)  0.001278   1.0  0.009482  0.922700
 Residual                     9.706003  72.0       NaN       NaN}

okay now do within-electrode anova too

In [611]:
def process_and_append_trial_data_for_anova(time_averaged_lists, conditions):
    """
    Transforms nested trial-level electrophysiological data into a flat list
    of dictionaries, suitable for creating a Pandas DataFrame for ANOVA.

    The function iterates through time windows, experimental conditions,
    Regions of Interest (ROIs), subjects, trials, and significant electrodes.
    For each trial of each electrode, it extracts the activity and compiles
    this along with subject, electrode, ROI, time window, trial number, and
    condition-specific parameters into a dictionary.

    Parameters:
    ----------
    time_averaged_lists : dict
        A nested dictionary containing the trial-level data. Despite the name,
        this function expects trial-by-trial data, not time-averaged data.
        Expected structure:
        `time_averaged_lists[time_window_name][condition_name][roi_name]`
        This path should lead to a list of NumPy arrays. Each array in this list
        represents data for one subject and is typically shaped
        `(n_trials, n_significant_electrodes_for_this_subject_in_this_roi)`.
        Example: `{'FirstHalfSecond': {'ConditionA': {'dlPFC': [subj1_trial_data, subj2_trial_data, ...]}}}`
        where `subj1_trial_data` would be `(num_trials, num_electrodes)`.
    conditions : dict
        A dictionary defining the experimental conditions and their parameters.
        Keys are condition names (e.g., 'Stimulus_cr') matching those in
        `time_averaged_lists`. Values are dictionaries containing the
        parameter names and their specific values for that condition, which will
        be added as columns in the output.
        Example: `{'Stimulus_cr': {'Congruency': 'Congruent', 'SwitchType': 'Repeat'}}`

    Returns:
    -------
    list of dict
        A list where each dictionary represents a single row for an ANOVA DataFrame,
        corresponding to a single trial on a single electrode.
        Each dictionary contains the following keys:
        - 'SubjectID': Subject identifier.
        - 'Electrode': Electrode name.
        - 'ROI': Region of Interest.
        - 'TimeWindow': The time window from which the data was processed.
        - 'Trial': The trial number (1-indexed).
        - 'Activity': The electrophysiological activity for that trial, electrode,
                      subject, ROI, time window, and condition.
        - Keys from `condition_parameters`: Additional columns based on the
          experimental design (e.g., 'Congruency', 'SwitchType').

    Notes:
    -----
    - This function assumes that `rois` (a list of ROI names) and
      `sig_electrodes_per_subject_roi` (a dict mapping
      `[roi_name][subject_id]` to a list of significant electrode names)
      are available in the scope where the function is called.
    - The number of items in the list `lists[condition][roi]` (i.e., number of subjects
      with data for that condition/ROI) must correspond to the order of subjects
      derived from `subjects_with_data` for correct SubjectID mapping.
    - Trials with NaN values or incorrect numbers of electrodes (not matching
      the count in `sig_electrodes_per_subject[subject_id]`) are skipped.
    """
    data_for_anova = []
    for time_window, lists in time_averaged_lists.items():
        for condition, condition_parameters in conditions.items():
            for roi in rois:
                sig_electrodes_per_subject = sig_electrodes_per_subject_roi[roi]
                subjects_with_data = [subject for subject, electrodes in sig_electrodes_per_subject.items() if electrodes] # Skip over subjects without data
                for subject_index, subject_data in enumerate(lists[condition][roi]):
                    subject_id = subjects_with_data[subject_index]

                    if subject_id not in sig_electrodes_per_subject or not sig_electrodes_per_subject[subject_id]:
                        continue

                    for trial_index, trial_data in enumerate(subject_data):
                        # Skip trials with any missing data or incorrect length
                        if np.any(np.isnan(trial_data)) or len(trial_data) != len(sig_electrodes_per_subject[subject_id]):
                            continue

                        for electrode_index, electrode_name in enumerate(sig_electrodes_per_subject[subject_id]):
                            activity = trial_data[electrode_index] if electrode_index < len(trial_data) else np.nan

                            # Prepare the data dictionary
                            data_dict = {
                                'SubjectID': subject_id,
                                'Electrode': electrode_name,
                                'ROI': roi,
                                'TimeWindow': time_window,
                                'Trial': trial_index + 1,
                                'Activity': activity
                            }

                            # Dynamically add condition types and their values
                            data_dict.update(condition_parameters)

                            data_for_anova.append(data_dict)
    return data_for_anova
# # Example usage with the `time_averaged_lists` dictionary
# time_averaged_lists = {
#     "FirstHalfSecond": output_data_timeAvg_firstHalfSecond_lists,
#     "SecondHalfSecond": output_data_timeAvg_secondHalfSecond_lists,
#     # "FullSecond": output_data_timeAvg_fullSecond_lists  # Uncomment or comment based on your needs
# }

data_for_anova = process_and_append_trial_data_for_anova(data_timeAvg_lists, conditions)

# Convert to DataFrame
df_for_trial_level_anova = pd.DataFrame(data_for_anova)


In [612]:
df_for_trial_level_anova

,SubjectID,Electrode,ROI,TimeWindow,Trial,Activity,BIDS_events,switchType
0,D0057,RAI6,lpfc,firstHalfSecond,1,0.038071,"[Stimulus/r25.0, Stimulus/r75.0]",r
1,D0057,RAMF14,lpfc,firstHalfSecond,1,0.835029,"[Stimulus/r25.0, Stimulus/r75.0]",r
2,D0057,RAI6,lpfc,firstHalfSecond,2,-0.329568,"[Stimulus/r25.0, Stimulus/r75.0]",r
3,D0057,RAMF14,lpfc,firstHalfSecond,2,0.741989,"[Stimulus/r25.0, Stimulus/r75.0]",r
4,D0057,RAI6,lpfc,firstHalfSecond,3,0.005482,"[Stimulus/r25.0, Stimulus/r75.0]",r
...,...,...,...,...,...,...,...,...
48789,D0103,LTPI3,occ,secondHalfSecond,161,-0.147525,"[Stimulus/s25.0, Stimulus/s75.0]",s
48790,D0103,LTPI4,occ,secondHalfSecond,161,0.301400,"[Stimulus/s25.0, Stimulus/s75.0]",s
48791,D0103,LTPI2,occ,secondHalfSecond,162,-0.118228,"[Stimulus/s25.0, Stimulus/s75.0]",s
48792,D0103,LTPI3,occ,secondHalfSecond,162,-0.386592,"[Stimulus/s25.0, Stimulus/s75.0]",s


In [613]:
# Assuming df_for_trial_level_anova is your DataFrame and it includes a 'SubjectID' column
def perform_modular_within_electrode_anova_roi(df, conditions, save_dir, save_name):
    """
    Performs a within-electrode ANOVA for each specified time window, subject,
    electrode, and ROI to identify significant experimental condition effects.

    This function iterates through each unique combination of subject, electrode,
    ROI, and time window present in the input DataFrame. For each combination,
    it conducts an ANOVA on the 'Activity' data, using experimental factors
    derived from the `conditions` dictionary. The ANOVA model includes main
    effects of these conditions and all their interaction terms.

    The function saves three types of output files:
    1.  A text file with all ANOVA tables for every combination.
    2.  A text file with only the significant ANOVA tables (where PR(>F) < 0.05).
    3.  A JSON file containing a structured dictionary of significant effects
        (effect name and p-value) organized by subject, electrode, ROI, and
        time window.

    Note: This function performs separate ANOVAs for each time window and does
    not test for interactions of experimental conditions *with* the time window factor.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing trial-level data. It must include
        columns: 'SubjectID', 'Electrode', 'TimeWindow', 'ROI', 'Activity'
        (as the dependent variable), and columns for each experimental
        factor specified in the `conditions` dictionary.
    conditions : dict
        A dictionary defining the experimental conditions. The keys of the
        inner dictionaries (e.g., `conditions['Stimulus_condition_A'].keys()`)
        are used as factor names in the ANOVA formula (e.g., 'congruency',
        'switchType'). 'BIDS_events' is excluded if present.
    rois : list
        A list of ROI names (strings) that are present in the 'ROI' column of `df`.
        Used for constructing the output filename suffix.
    save_dir : str
        The absolute path to the directory where the output files will be saved.
    save_name : str
        A base name used as a prefix for the output filenames.

    Returns
    -------
    tuple
        A tuple containing three elements:
        - results (list of dict): Each dictionary contains 'SubjectID',
          'Electrode', 'TimeWindow', 'ROI', and the full 'ANOVA_Results'
          table (statsmodels anova_lm object) for that combination.
        - significant_results (list of dict): Similar to `results`, but only
          includes entries where at least one effect was significant.
          The 'SignificantEffects' key holds the subset of the ANOVA table
          with PR(>F) < 0.05.
        - significant_effects_structure (dict): A nested dictionary where
          keys are subject IDs, then electrode names, then ROI names, then
          time window names, and values are dictionaries of significant
          effect names and their p-values (e.g., {'congruency': 0.04}).
    """
    results = []
    significant_effects_structure = {}

    for subject_id in df['SubjectID'].unique():
        for electrode in df['Electrode'].unique():
            for time_window in df['TimeWindow'].unique():
                for roi in df['ROI'].unique():
                    df_filtered = df[(df['SubjectID'] == subject_id) & 
                                    (df['Electrode'] == electrode) & 
                                    (df['TimeWindow'] == time_window) &
                                    (df['ROI'] == roi)]
                    
                    if df_filtered.empty: #if this combination of subject, electrode, and time window doesn't exist, then move on.
                        continue

                    # Dynamically construct the formula based on condition keys present in the DataFrame
                    condition_keys = [key for key in conditions[next(iter(conditions))].keys() if key != 'BIDS_events']
                    formula_terms = ' + '.join([f'C({name})' for name in condition_keys])
                    interaction_terms = ' * '.join([f'C({name})' for name in condition_keys])
                    formula = f'Activity ~ {formula_terms} + {interaction_terms}'

                    # Perform the ANOVA
                    model = ols(formula, data=df_filtered).fit()
                    anova_results = anova_lm(model, typ=2)
                    
                    # Append the results
                    results.append({
                        'SubjectID': subject_id,
                        'Electrode': electrode,
                        'TimeWindow': time_window,
                        'ROI': roi,
                        'ANOVA_Results': anova_results
                    })

    # Join all the ROIs with '_' as the separator
    rois_suffix = '_'.join(rois)

    # Add the suffix '_onlySigElectrodes' to the base filename
    allElectrodesFilename = f"{save_name}_allElectrodes_{rois_suffix}.txt"
    onlySigElectrodesFilename = f"{save_name}_onlySigElectrodes_{rois_suffix}.txt"
    significantEffectsStructureFilename = f"{save_name}_significantEffectsStructure_{rois_suffix}.txt"

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, allElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"results saved to: {results_file_path}")

    # Now process the significant results, including the subject ID in the output
    significant_results = []

    for result in results:
        anova_table = result['ANOVA_Results']
        subject_id = result['SubjectID']
        electrode = result['Electrode']
        time_window = result['TimeWindow']
        roi = result['ROI']
        significant_effects = anova_table[anova_table['PR(>F)'] < 0.05]
        
        if not significant_effects.empty:
            print(f"Significant effects found for Subject: {subject_id}, Electrode: {electrode}, Time Window: {time_window}, ROI: {roi}")
            print(significant_effects)
            print("\n")
            
            significant_results.append({
                'SubjectID': subject_id,
                'Electrode': electrode,
                'TimeWindow': time_window,
                'ROI': roi,
                'SignificantEffects': significant_effects
            })

        # Extract significant effects for the current result. Basically just get the p-value. 3/19.
        sig_effects_just_p_values = utils.extract_significant_effects(anova_table)
        
        if sig_effects_just_p_values:
            # Ensure subject_id and electrode keys exist
            if subject_id not in significant_effects_structure:
                significant_effects_structure[subject_id] = {}
            if electrode not in significant_effects_structure[subject_id]:
                significant_effects_structure[subject_id][electrode] = {}
            if electrode not in significant_effects_structure[subject_id][electrode]:
                significant_effects_structure[subject_id][electrode][roi] = {}

            # Assign the significant effects and their p-values to the correct structure
            significant_effects_structure[subject_id][electrode][roi][time_window] = sig_effects_just_p_values    

    # Define the full path for the results file
    significant_results_file_path = os.path.join(save_dir, onlySigElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(significant_results_file_path, 'w') as file:
        file.write(significant_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_results saved to: {significant_results_file_path}")

    significant_effects_structure_file_path = os.path.join(save_dir, significantEffectsStructureFilename)
    # Save the ANOVA results to a json file (if this works, change the others to json files too)
    with open(significant_effects_structure_file_path, 'w') as file:
        json.dump(significant_effects_structure, file, indent=4)

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_effects_structure saved to: {significant_effects_structure_file_path}")

    return results, significant_results, significant_effects_structure

In [614]:
# Assuming df_for_trial_level_anova is your DataFrame and it includes a 'SubjectID' column
def perform_modular_within_electrode_anova_roi_timeWindowInteractions(df, conditions, save_dir, save_name):
    """
    Performs a within-electrode ANOVA including 'TimeWindow' as a factor.

    This function iterates through each unique combination of subject, electrode,
    and ROI. For each combination, it conducts an ANOVA on the 'Activity' data.
    The model includes:
    - Main effects of experimental conditions (derived from `conditions` dict).
    - Main effect of 'TimeWindow'.
    - All interaction terms between experimental conditions and 'TimeWindow'.

    This allows testing if overall activity or condition-specific activity
    differs significantly across different time windows for each electrode.
    The output ANOVA table will indicate significance for terms like
    'C(TimeWindow)', 'C(condition_A):C(TimeWindow)', etc.

    The function saves three types of output files:
    1.  A text file with all ANOVA tables for every subject/electrode/ROI combination.
    2.  A text file with only the significant ANOVA tables (where PR(>F) < 0.05 for any term).
    3.  A JSON file containing a structured dictionary of significant effects
        (effect name and p-value) organized by subject, electrode, and ROI.
        (Note: TimeWindow is a factor in the ANOVA, not a level in this output structure).

    Dependencies:
    - Assumes 'rois' (a list of strings, e.g., ['lpfc', 'acc']) is defined in the
      calling scope and contains the relevant ROI names for constructing filenames.
    - Assumes `utils.extract_significant_effects` function is available for
      processing ANOVA tables.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing trial-level data. Must include
        columns: 'SubjectID', 'Electrode', 'ROI', 'TimeWindow', 'Activity'
        (as the dependent variable), and columns for each experimental
        factor specified in the `conditions` dictionary.
    conditions : dict
        A dictionary defining the experimental conditions. The keys of the
        inner dictionaries (e.g., `conditions['Stimulus_condition_A'].keys()`)
        are used as factor names in the ANOVA formula (e.g., 'congruency',
        'switchType'). 'BIDS_events' is excluded if present.
    save_dir : str
        The absolute path to the directory where the output files will be saved.
    save_name : str
        A base name used as a prefix for the output filenames.

    Returns
    -------
    tuple
        A tuple containing three elements:
        - results (list of dict): Each dictionary contains 'SubjectID',
          'Electrode', 'ROI', and the full 'ANOVA_Results' table
          (statsmodels anova_lm object) for that combination. This table
          will include effects related to 'TimeWindow'.
        - significant_results (list of dict): Similar to `results`, but only
          includes entries where at least one effect in the ANOVA table was
          significant. The 'SignificantEffects' key holds the subset of the
          ANOVA table with PR(>F) < 0.05.
        - significant_effects_structure (dict): A nested dictionary where
          keys are subject IDs, then electrode names, then ROI names.
          The values are dictionaries of significant effect names (which can
          include 'TimeWindow' or its interactions) and their p-values
          (e.g., {'C(TimeWindow)': 0.03, 'congruency:C(TimeWindow)': 0.04}).
    """
    results = []
    significant_effects_structure = {}

    for subject_id in df['SubjectID'].unique():
        for electrode in df['Electrode'].unique():
            for roi in df['ROI'].unique():
                df_filtered = df[(df['SubjectID'] == subject_id) & 
                                    (df['Electrode'] == electrode) & (df['ROI'] == roi)]
                
                if df_filtered.empty:
                    continue
                
                # Dynamically construct the formula based on condition keys present in the DataFrame, skipping BIDS_events
                condition_keys = [key for key in conditions[next(iter(conditions))].keys() if key != 'BIDS_events']
                formula_terms = ' + '.join([f'C({name})' for name in condition_keys] + ['C(TimeWindow)']) # maybe add roi too? 5/20.
                interaction_terms = ' * '.join([f'C({name})' for name in condition_keys] + ['C(TimeWindow)'])
                formula = f'Activity ~ {formula_terms} + {interaction_terms}'

                # Perform the ANOVA
                model = ols(formula, data=df_filtered).fit()
                anova_results = anova_lm(model, typ=2)
                
                # Append the results
                results.append({
                    'SubjectID': subject_id,
                    'Electrode': electrode,
                    'ROI': roi,
                    'ANOVA_Results': anova_results
                })
    
    # Join all the ROIs with '_' as the separator
    rois_suffix = '_'.join(rois)

    # Add the suffix '_onlySigElectrodes' to the base filename
    allElectrodesFilename = f"{save_name}_allElectrodes_{rois_suffix}.txt"
    onlySigElectrodesFilename = f"{save_name}_onlySigElectrodes_{rois_suffix}.txt"
    significantEffectsStructureFilename = f"{save_name}_significantEffectsStructure_timeWindowInteractions_{rois_suffix}.txt"

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, allElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"results saved to: {results_file_path}")

    # Now process the significant results, including the subject ID in the output
    significant_results = []

    for result in results:
        anova_table = result['ANOVA_Results']
        subject_id = result['SubjectID']
        electrode = result['Electrode']
        roi = result['ROI']
        
        significant_effects = anova_table[anova_table['PR(>F)'] < 0.05]
        
        if not significant_effects.empty:
            print(f"Significant effects found for Subject: {subject_id}, Electrode: {electrode}, ROI: {roi}")
            print(significant_effects)
            print("\n")
            
            significant_results.append({
                'SubjectID': subject_id,
                'Electrode': electrode,
                'ROI': roi,
                'SignificantEffects': significant_effects
            })

        # Extract significant effects for the current result. Basically just get the p-value. 3/19.
        sig_effects_just_p_values = utils.extract_significant_effects(anova_table)
        
        if sig_effects_just_p_values:
            # Ensure subject_id and electrode keys exist
            if subject_id not in significant_effects_structure:
                significant_effects_structure[subject_id] = {}
            if electrode not in significant_effects_structure[subject_id]:
                significant_effects_structure[subject_id][electrode] = {}
            if electrode not in significant_effects_structure[subject_id][electrode]:
                significant_effects_structure[subject_id][electrode][roi] = {}

            # Assign the significant effects and their p-values to the correct structure
            significant_effects_structure[subject_id][electrode][roi] = sig_effects_just_p_values    

    # Define the full path for the results file
    significant_results_file_path = os.path.join(save_dir, onlySigElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(significant_results_file_path, 'w') as file:
        file.write(significant_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_results saved to: {significant_results_file_path}")

    significant_effects_structure_file_path = os.path.join(save_dir, significantEffectsStructureFilename)
    # Save the ANOVA results to a json file (if this works, change the others to json files too)
    with open(significant_effects_structure_file_path, 'w') as file:
        json.dump(significant_effects_structure, file, indent=4)

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_effects_structure saved to: {significant_effects_structure_file_path}")

    return results, significant_results, significant_effects_structure


run within electrode anova

In [615]:
results, significant_results, significant_effects_structure = perform_modular_within_electrode_anova_roi(df_for_trial_level_anova, conditions, save_dir, conditions_save_name)

results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\stimulus_switch_type_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_17_subjects_allElectrodes_lpfc_occ.txt
Significant effects found for Subject: D0057, Electrode: RAMF14, Time Window: firstHalfSecond, ROI: lpfc
                 sum_sq   df          F    PR(>F)
C(switchType)  2.887388  1.0  10.649319  0.001196


Significant effects found for Subject: D0059, Electrode: LMMF11, Time Window: secondHalfSecond, ROI: lpfc
                sum_sq   df        F    PR(>F)
C(switchType)  3.31639  1.0  6.20765  0.013103


Significant effects found for Subject: D0059, Electrode: LMMF10, Time Window: secondHalfSecond, ROI: lpfc
                  sum_sq   df          F        PR(>F)
C(switchType)  13.903568  1.0  24.652536  9.984627e-07


Significant effects found for Subject: D0063, Electrode

4/4 do both perform_modular_within_electrode_anova_roi and perform_modular_within_electrode_anova_roi_timeWindowInteractions. This will tell us which time windows have significant activity in an electrode AND if there is significant differences in activity across time windows in an electrode.

In [616]:
results_timeWindowInteraction, significant_results_timeWindowInteraction, significant_effects_structure_timeWindowInteraction = perform_modular_within_electrode_anova_roi_timeWindowInteractions(df_for_trial_level_anova, conditions, save_dir, conditions_save_name)

results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\stimulus_switch_type_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_17_subjects_allElectrodes_lpfc_occ.txt
Significant effects found for Subject: D0057, Electrode: RAI6, ROI: lpfc
                 sum_sq   df          F        PR(>F)
C(TimeWindow)  8.313221  1.0  33.233456  1.167798e-08


Significant effects found for Subject: D0057, Electrode: RAMF14, ROI: lpfc
                 sum_sq   df         F    PR(>F)
C(switchType)  2.490223  1.0  8.646562  0.003371


Significant effects found for Subject: D0059, Electrode: LMMF11, ROI: lpfc
                  sum_sq   df           F        PR(>F)
C(switchType)   1.721430  1.0    3.995719  4.593625e-02
C(TimeWindow)  54.998865  1.0  127.661349  1.151465e-27


Significant effects found for Subject: D0059, Electrode: LMMF10, ROI: lpfc
    

In [617]:
results

[{'SubjectID': 'D0057',
  'Electrode': 'RAI6',
  'TimeWindow': 'firstHalfSecond',
  'ROI': 'lpfc',
  'ANOVA_Results':                   sum_sq     df         F    PR(>F)
  C(switchType)   0.006385    1.0  0.034609  0.852511
  Residual       73.614053  399.0       NaN       NaN},
 {'SubjectID': 'D0057',
  'Electrode': 'RAI6',
  'TimeWindow': 'secondHalfSecond',
  'ROI': 'lpfc',
  'ANOVA_Results':                    sum_sq     df        F    PR(>F)
  C(switchType)    0.083816    1.0  0.26541  0.606713
  Residual       126.002574  399.0      NaN       NaN},
 {'SubjectID': 'D0057',
  'Electrode': 'RAMF14',
  'TimeWindow': 'firstHalfSecond',
  'ROI': 'lpfc',
  'ANOVA_Results':                    sum_sq     df          F    PR(>F)
  C(switchType)    2.887388    1.0  10.649319  0.001196
  Residual       108.182284  399.0        NaN       NaN},
 {'SubjectID': 'D0057',
  'Electrode': 'RAMF14',
  'TimeWindow': 'secondHalfSecond',
  'ROI': 'lpfc',
  'ANOVA_Results':                    sum_sq     

In [618]:
significant_results 

[{'SubjectID': 'D0057',
  'Electrode': 'RAMF14',
  'TimeWindow': 'firstHalfSecond',
  'ROI': 'lpfc',
  'SignificantEffects':                  sum_sq   df          F    PR(>F)
  C(switchType)  2.887388  1.0  10.649319  0.001196},
 {'SubjectID': 'D0059',
  'Electrode': 'LMMF11',
  'TimeWindow': 'secondHalfSecond',
  'ROI': 'lpfc',
  'SignificantEffects':                 sum_sq   df        F    PR(>F)
  C(switchType)  3.31639  1.0  6.20765  0.013103},
 {'SubjectID': 'D0059',
  'Electrode': 'LMMF10',
  'TimeWindow': 'secondHalfSecond',
  'ROI': 'lpfc',
  'SignificantEffects':                   sum_sq   df          F        PR(>F)
  C(switchType)  13.903568  1.0  24.652536  9.984627e-07},
 {'SubjectID': 'D0063',
  'Electrode': 'LPSF12',
  'TimeWindow': 'firstHalfSecond',
  'ROI': 'lpfc',
  'SignificantEffects':                  sum_sq   df         F    PR(>F)
  C(switchType)  1.175162  1.0  5.553966  0.018946},
 {'SubjectID': 'D0063',
  'Electrode': 'RAI5',
  'TimeWindow': 'secondHalfSecond

### plot and QC stats

plot time perm cluster stats (don't run this immediately below cell if didn't do time perm cluster)

In [619]:
# # Plotting
# plt.figure(figsize=(10, 6))
# plt.plot(time_perm_cluster_results['dlpfc'])
# plt.xlabel('Timepoints')
# plt.ylabel('Significance (0 or 1)')
# plt.title('Permutation Test Significance Over Time')
# plt.show()

### plot interaction effects (only do this when load in all four of them)

https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
# # add the other conditions and give them condition names and colors too
plotting_parameters = {
    'Stimulus_r': {
        'condition_parameter': 'repeat',
        'color': 'blue',
        "line_style": "-"
    },
    'Stimulus_s': {
        'condition_parameter': 'switch',
        'color': 'blue',
        "line_style": "--"
    },
    'Stimulus_c': {
        'condition_parameter': 'congruent',
        'color': 'red',
        "line_style": "-"
    },
    'Stimulus_i': {
        'condition_parameter': 'incongruent',
        'color': 'red',
        "line_style": "--"
    },
    "Stimulus_ir": {
        "condition_parameter": "IR",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_is": {
        "condition_parameter": "IS",
        "color": "blue",
        "line_style": "--"
    },
    "Stimulus_cr": {
        "condition_parameter": "CR",
        "color": "red",
        "line_style": "-"
    },
    "Stimulus_cs": {
        "condition_parameter": "CS",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_c25": {
        "condition_parameter": "c75",
        "color": "pink",
        "line_style": "-"
    },
    "Stimulus_c75": {
        "condition_parameter": "c25",
        "color": "orange",
        "line_style": "-"
    },
    "Stimulus_i25": {
        "condition_parameter": "i75",
        "color": "pink",
        "line_style": "--"
    },
    "Stimulus_i75": {
        "condition_parameter": "i25",
        "color": "orange",
        "line_style": "--"
    },
    "Stimulus_s25": {
        "condition_parameter": "s25",
        "color": "skyblue",
        "line_style": "--"
    },
    "Stimulus_s75": {
        "condition_parameter": "s75",
        "color": "purple",
        "line_style": "--"
    },
    "Stimulus_r25": {
        "condition_parameter": "r25",
        "color": "skyblue",
        "line_style": "-"
    },
    "Stimulus_r75": {
        "condition_parameter": "r75",
        "color": "purple",
        "line_style": "-"
    },
    "Stimulus_bigH": {
        "condition_parameter": "bigH",
        "color": "green",
        "line_style": "-"
    },
    "Stimulus_bigS": {
        "condition_parameter": "bigS",
        "color": "green",
        "line_style": "--"
    },
    "Stimulus_smallH": {
        "condition_parameter": "smallH",
        "color": "orange",
        "line_style": "-"
    },
    "Stimulus_smallS": {
        "condition_parameter": "smallS",
        "color": "orange",
        "line_style": "--"
    },
    "Stimulus_taskG": {
        "condition_parameter": "taskG",
        "color": "gray",
        "line_style": "-"
    },
    "Stimulus_taskL": {
        "condition_parameter": "taskL",
        "color": "gray",
        "line_style": "--"
    },
}

copy the same plotting parameters for response as stimulus

In [ ]:
# Loop to add 'Response' keys
for key, value in list(plotting_parameters.items()):
    if key.startswith('Stimulus'):
        new_key = key.replace('Stimulus', 'Response')
        plotting_parameters[new_key] = value

# Optional: printing to verify
for key, value in plotting_parameters.items():
    print(f"{key}: {value}")


Stimulus_r: {'condition_parameter': 'repeat', 'color': 'blue', 'line_style': '-'}
Stimulus_s: {'condition_parameter': 'switch', 'color': 'blue', 'line_style': '--'}
Stimulus_c: {'condition_parameter': 'congruent', 'color': 'red', 'line_style': '-'}
Stimulus_i: {'condition_parameter': 'incongruent', 'color': 'red', 'line_style': '--'}
Stimulus_ir: {'condition_parameter': 'IR', 'color': 'blue', 'line_style': '-'}
Stimulus_is: {'condition_parameter': 'IS', 'color': 'blue', 'line_style': '--'}
Stimulus_cr: {'condition_parameter': 'CR', 'color': 'red', 'line_style': '-'}
Stimulus_cs: {'condition_parameter': 'CS', 'color': 'red', 'line_style': '--'}
Stimulus_c25: {'condition_parameter': 'c75', 'color': 'pink', 'line_style': '-'}
Stimulus_c75: {'condition_parameter': 'c25', 'color': 'orange', 'line_style': '-'}
Stimulus_i25: {'condition_parameter': 'i75', 'color': 'pink', 'line_style': '--'}
Stimulus_i75: {'condition_parameter': 'i25', 'color': 'orange', 'line_style': '--'}
Stimulus_s25: {'co

In [623]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
condition_names = list(conditions.keys()) # get the condition names as a list

def plot_interact_effects_modular_roi(roi, save_dir, save_name, mean_and_sem, condition_names, plotting_parameters, font_size=14):
    # Set global font size
    plt.rcParams.update({'font.size': font_size})

    # Base setup for directories and file paths
    save_path = os.path.join(save_dir, f'avg_{roi}_{save_name}_interactEffects_power_zscore_roi.png')

    # Initialize plot
    plt.figure(figsize=(10, 6))

    # Dynamically select the first subject and use it to extract times
    first_subject_id = next(iter(subjects_mne_objects))
    example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
    times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

    overall_averages_for_plotting = {}
    overall_sem_for_plotting = {}
    # Initialize variables to store the global min and max values
    global_min_val = float('inf')  # Set to infinity initially
    global_max_val = float('-inf')  # Set to negative infinity initially
    
    # Generate labels and plot each condition
    for index, condition_name in enumerate(condition_names):
        overall_averages_for_plotting[condition_name] = mean_and_sem[roi][condition_name]['mean']
        overall_sem_for_plotting[condition_name] = mean_and_sem[roi][condition_name]['sem']

        # Calculate the minimum value for this condition, including SEM
        current_min_val = min(overall_averages_for_plotting[condition_name] - overall_sem_for_plotting[condition_name])
        # Calculate the maximum value for this condition, including SEM
        current_max_val = max(overall_averages_for_plotting[condition_name] + overall_sem_for_plotting[condition_name])

        # Update the global min and max values if necessary
        global_min_val = min(global_min_val, current_min_val)
        global_max_val = max(global_max_val, current_max_val)

        # Optionally, add a small margin to the range
        margin = (global_max_val - global_min_val) * 0.05  # 5% of the range as margin
        global_min_val -= margin
        global_max_val += margin

        label = plotting_parameters[condition_name]['condition_parameter']  # extract label from plotting parameters dict
        color = plotting_parameters[condition_name]['color']
        line_style = plotting_parameters[condition_name]['line_style']

        plt.plot(times, overall_averages_for_plotting[condition_name], label=f'Average {roi} {label}', linestyle=line_style, color=color)
        plt.fill_between(times, overall_averages_for_plotting[condition_name] - overall_sem_for_plotting[condition_name], overall_averages_for_plotting[condition_name] + overall_sem_for_plotting[condition_name], alpha=0.3, color=color)

    plt.xlabel('Time (s)')
    plt.ylabel('Z-score Power')
    plt.title(f'Average {roi} signal for {save_name}')
    plt.legend()

    # Adjust the y-axis limits
    plt.ylim([global_min_val, global_max_val])

    # Remove the top and right borders (spines)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.savefig(save_path)
    plt.close()

plot average traces

In [624]:
for roi in rois:
    plot_interact_effects_modular_roi(roi, save_dir, conditions_save_name, mean_and_sem, condition_names, plotting_parameters)

try to plot different groups of electrodes based on significance for effects 4/4  

In [ ]:
def get_significant_electrodes_by_effect(effect_structure, effect_type):
    '''
    Extracts electrodes with significant specified effects.

    :param effect_structure: Dictionary containing significant effects for each electrode, subject, and roi.
    :param effect_type: Single string or list of strings specifying the effect(s) of interest ("congruency", "switchType", "congruency:switchType", etc).
    :return: A dictionary mapping subject IDs to lists of electrodes with the specified significant effect(s).
    '''
    significant_electrodes = {}

    # Ensure effect_type is a list to simplify the logic
    if isinstance(effect_type, str):
        effect_type = [effect_type]

    # Loop over each subject in the effect structure
    for subject_id, electrodes in effect_structure.items():
        significant_electrodes_for_subject = []  # Temporary list to hold significant electrodes for the current subject

        # Loop over each electrode and its associated ROIs
        for electrode, rois in electrodes.items():
            for roi, effects in rois.items():
                # Loop over each effect and p-value tuple
                for effect, p_value in effects:
                    if effect in effect_type and p_value < 0.05:  # Check if the effect is of interest and is significant
                        significant_electrodes_for_subject.append(electrode)
                        break  # Stop checking further effects for this electrode once a significant effect is found

        # Add the subject's significant electrodes to the result if any were found
        if significant_electrodes_for_subject:
            significant_electrodes[subject_id] = significant_electrodes_for_subject

    return significant_electrodes


this is for congruency x switch type interaction

In [ ]:
condition_names

In [ ]:
significant_effects_structure_timeWindowInteraction

In [ ]:
congruencySigElectrodes

In [ ]:
if 'Stimulus_cr' in condition_names:
    congruencySigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruency')
    switchTypeSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'switchType')
    congruencySwitchTypeInteractionSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruency:switchType')
    allEffectSensitiveElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, ['congruency', 'switchType', 'congruency:switchType'])

In [ ]:
congruencySigElectrodes

In [ ]:
switchTypeSigElectrodes

In [ ]:
congruencySwitchTypeInteractionSigElectrodes

In [ ]:
allEffectSensitiveElectrodes

this one is for congruency x congruency proportion interaction

In [ ]:
if 'Stimulus_c25' in condition_names:
    congruencySigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruency')
    congruencyProportionSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruencyProportion')
    congruencyCongruencyProportionInteractionSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'congruency:congruencyProportion')
    allEffectSensitiveElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, ['congruency', 'congruencyProportion', 'congruency:congruencyProportion'])

this is for switch type x switch proportion interaction

In [ ]:
if 'Stimulus_s25' in condition_names:
    switchTypeSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'switchType')
    switchProportionSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'switchProportion')
    switchTypeSwitchProportionInteractionSigElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, 'switchType:switchProportion')
    allEffectSensitiveElectrodes = get_significant_electrodes_by_effect(significant_effects_structure_timeWindowInteraction, ['switchType', 'switchProportion', 'switchType:switchProportion'])

remake overall averages but just for the chosen electrodes. This should probably just be incorporated into process_data_for_roi (can pass in sig_electrodes as an input) but too lazy to do that right now..5/20.

In [ ]:
# Time windows

original_time_indices = {
    'firstHalfSecond': (2048, 3072),
    'secondHalfSecond': (3072, 4096)
}

sampling_rate_ratio = sampling_rate / 2048.0 # account for decimation factor

# Convert the time indices to the new sampling rate
time_indices = {
    key: (int(start * sampling_rate_ratio), int(end * sampling_rate_ratio))
    for key, (start, end) in original_time_indices.items()
}

start_idx_firstHalfSecond, end_idx_firstHalfSecond = time_indices['firstHalfSecond']
start_idx_secondHalfSecond, end_idx_secondHalfSecond = time_indices['secondHalfSecond']
start_idx_fullSecond, end_idx_fullSecond = start_idx_firstHalfSecond, end_idx_secondHalfSecond

def is_dict_of_dicts(d):
    """Check if the input is a dictionary of dictionaries."""
    return isinstance(d, dict) and all(isinstance(val, dict) for val in d.values())

def get_sig_electrodes(sig_electrodes, roi, sub):
    """Get significant electrodes based on the structure of sig_electrodes. Significant effects structure removes roi info, but sig_electrodes_per_subject_roi keeps it.
    maybe I should make these consistent... 4/5"""
    if is_dict_of_dicts(sig_electrodes):
        return sig_electrodes[roi].get(sub, [])
    else:
        return sig_electrodes.get(sub, [])
    
# this massive function needs to be split up and replace its above non-function form once it works 4/5.
def get_average_data_for_specific_electrodes(subjects, rois, condition_names, sig_electrodes):
    # Assuming condition_names contains all four conditions
    # Initializing dictionaries
    output_data_trialAvg_lists = utils.initialize_output_data(rois, condition_names)
    output_data_trialStd_lists = utils.initialize_output_data(rois, condition_names)
    output_data_timeAvg_firstHalfSecond_lists = utils.initialize_output_data(rois, condition_names)
    output_data_timeAvg_secondHalfSecond_lists = utils.initialize_output_data(rois, condition_names)
    output_data_timeAvg_fullSecond_lists = utils.initialize_output_data(rois, condition_names)

    # Initialize a dictionary to hold mappings
    overall_electrode_mapping = []

    # Initialize a dictionary to hold mappings for each ROI
    electrode_mapping_per_roi = {roi: [] for roi in rois}
    for sub in subjects:
        for roi in rois:
            for condition_name in condition_names:
                # Determine significant electrodes for the current ROI and subject. Hmm not sure if this will work if I have more than one ROI for the 
                # get_significant_electrodes_by_effect electrodes.. 4/5
                sig_electrodes_this_sub = get_sig_electrodes(sig_electrodes, roi, sub)
                print('sub', sub)
                print('sig elecs:', sig_electrodes_this_sub)
                
                if not sig_electrodes_this_sub:  # Skip if no significant electrodes
                    continue
                            
                for electrode in sig_electrodes_this_sub:
                    # For each significant electrode, append a tuple to the mapping list
                    # Tuple format: (Subject ID, ROI, Electrode Name, Index in List)
                    # The index can be the current length of the list before appending
                    index = len(overall_electrode_mapping)
                    overall_electrode_mapping.append((sub, roi, electrode, condition_name, index))  

                    # For each significant electrode, append a tuple to the mapping list of the corresponding ROI
                    # Tuple format: (Subject ID, Electrode Name, Index in List for this ROI)
                    index = len(electrode_mapping_per_roi[roi])  # Get the current length of the list for this ROI
                    electrode_mapping_per_roi[roi].append((sub, electrode, condition_name, index))
                    
                # Load trial-level data for the current condition and pick significant electrodes
                epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick_channels(sig_electrodes_this_sub)
                # print(epochs.get_data().shape)
                # Calculate averages for each time window
                trial_avg, trial_std, time_avg_firstHalfSecond = utils.filter_and_average_epochs(epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
                _, _, time_avg_secondHalfSecond = utils.filter_and_average_epochs(epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
                _, _, time_avg_fullSecond = utils.filter_and_average_epochs(epochs, start_idx_fullSecond, end_idx_fullSecond)
                print('time avg full second shape:', time_avg_fullSecond.shape)

                # Append the results to their respective lists
                output_data_trialAvg_lists[condition_name][roi].append(trial_avg)
                output_data_trialStd_lists[condition_name][roi].append(trial_std)
                output_data_timeAvg_firstHalfSecond_lists[condition_name][roi].append(time_avg_firstHalfSecond)
                output_data_timeAvg_secondHalfSecond_lists[condition_name][roi].append(time_avg_secondHalfSecond)
                output_data_timeAvg_fullSecond_lists[condition_name][roi].append(time_avg_fullSecond)


    # After collecting all data, concatenate across subjects for each roi and condition
    concatenated_trialAvg_data = {}
    concatenated_trialStd_data = {}

    for roi in rois:
        concatenated_trialAvg_data[roi] = {}
        concatenated_trialStd_data[roi] = {}

        for condition_name in condition_names:
            concatenated_trialAvg_data[roi][condition_name] = np.concatenate(output_data_trialAvg_lists[condition_name][roi], axis=0)
            concatenated_trialStd_data[roi][condition_name] = np.concatenate(output_data_trialStd_lists[condition_name][roi], axis=0)


    # Calculate mean and SEM across electrodes for all time windows and rois
    overall_averages = {}
    overall_sems = {}
    mean_and_sem = {roi: {condition_name: {} for condition_name in condition_names} for roi in rois}

    for roi in rois:
        overall_averages[roi] = {}
        overall_sems[roi] = {}
        for condition_name in condition_names:
            trialAvg_data = concatenated_trialAvg_data[roi][condition_name]
            overall_averages[roi][condition_name] = np.nanmean(trialAvg_data, axis=0)
            overall_sems[roi][condition_name] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])
            mean_and_sem[roi][condition_name] = {'mean': overall_averages[roi][condition_name], 'sem': overall_sems[roi][condition_name]}

    return concatenated_trialAvg_data, concatenated_trialStd_data, mean_and_sem, output_data_timeAvg_fullSecond_lists

this is for congruency x switch type interaction

In [ ]:
if 'Stimulus_cr' in condition_names:
    concatenated_trialAvg_data_congruencySigElectrodes, concatenated_trialStd_data_congruencySigElectrodes, congruencySigElectrodesMeanAndSEM, congruencySigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, congruencySigElectrodes)
    concatenated_trialAvg_data_switchTypeSigElectrodes, concatenated_trialStd_data_switchTypeSigElectrodes, switchTypeSigElectrodesMeanAndSEM, switchTypeSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, switchTypeSigElectrodes)
    concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes, concatenated_trialStd_data_congruencySwitchTypeInteractionSigElectrodes, congruencySwitchTypeInteractionSigElectrodesMeanAndSEM, congruencySwitchTypeInteractionSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, congruencySwitchTypeInteractionSigElectrodes)
    concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes, concatenated_trialStd_data_congruencySwitchTypeInteractionSigElectrodes, allEffectSensitiveElectrodesMeanAndSEM, allEffectSensitiveElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, allEffectSensitiveElectrodes)

this is for congruency x congruency proportion interaction

In [ ]:
if 'Stimulus_c25' in condition_names:
    concatenated_trialAvg_data_congruencySigElectrodes, concatenated_trialStd_data_congruencySigElectrodes, congruencySigElectrodesMeanAndSEM, congruencySigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, congruencySigElectrodes)
    concatenated_trialAvg_data_congruencyProportionSigElectrodes, concatenated_trialStd_data_congruencyProportionSigElectrodes, congruencyProportionSigElectrodesMeanAndSEM, congruencyProportionSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, congruencyProportionSigElectrodes)
    concatenated_trialAvg_data_congruencyCongruencyProportionInteractionSigElectrodes, concatenated_trialStd_data_congruencyCongruencyProportionInteractionSigElectrodes, congruencyCongruencyProportionInteractionSigElectrodesMeanAndSEM, congruencyCongruencyProportionInteractionSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, congruencyCongruencyProportionInteractionSigElectrodes)
    concatenated_trialAvg_data_allEffectSensitiveSigElectrodes, concatenated_trialStd_data_allEffectSensitiveSigElectrodes, allEffectSensitiveElectrodesMeanAndSEM, allEffectSensitiveElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, allEffectSensitiveElectrodes)

this for switch type x switch proportion interaction

In [ ]:
if 'Stimulus_s25' in condition_names:
    concatenated_trialAvg_data_switchTypeSigElectrodes, concatenated_trialStd_data_switchTypeSigElectrodes, switchTypeSigElectrodesMeanAndSEM, switchTypeSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, switchTypeSigElectrodes)
    concatenated_trialAvg_data_switchProportionSigElectrodes, concatenated_trialStd_data_switchProportionSigElectrodes, switchProportionSigElectrodesMeanAndSEM, switchProportionSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, switchProportionSigElectrodes)
    concatenated_trialAvg_data_switchTypeSwitchProportionInteractionSigElectrodes, concatenated_trialStd_data_switchTypeSwitchProportionInteractionSigElectrodes, switchTypeSwitchProportionInteractionSigElectrodesMeanAndSEM, switchTypeSwitchProportionInteractionSigElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, switchTypeSwitchProportionInteractionSigElectrodes)
    concatenated_trialAvg_data_allEffectSensitiveSigElectrodes, concatenated_trialStd_data_allEffectSensitiveSigElectrodes, allEffectSensitiveElectrodesMeanAndSEM, allEffectSensitiveElectrodes_timeAvg_fullSecondLists = get_average_data_for_specific_electrodes(subjects, rois, condition_names, allEffectSensitiveElectrodes)

finally plot the average traces for each output name for these chosen electrodes (this is for congruency x switch type)

*NOTE: This code only works for one roi. So go back and rerun everything with just one ROI for now. And fix this later (or just redo it all) 10/1.

In [ ]:
if len(rois) > 1:
    raise AssertionError("This conflates multiple ROIs right now, just load in one ROI at a time. Sorry, 5/20. Make better later.")
elif 'Stimulus_cr' in condition_names:
    plot_interact_effects_modular_roi(f'{rois[0]}', save_dir, 'congruency_switchType_congruencySigElectrodes', congruencySigElectrodesMeanAndSEM, condition_names, plotting_parameters)

In [ ]:
if len(rois) > 1:
    raise AssertionError("This conflates multiple ROIs right now, just load in one ROI at a time. Sorry, 5/20. Make better later.")
if 'Stimulus_cr' in condition_names:
    plot_interact_effects_modular_roi(f'{rois[0]}', save_dir, 'congruency_switchType_switchTypeSigElectrodes', switchTypeSigElectrodesMeanAndSEM, condition_names, plotting_parameters)

In [ ]:
if len(rois) > 1:
    raise AssertionError("This conflates multiple ROIs right now, just load in one ROI at a time. Sorry, 5/20. Make better later.")
if 'Stimulus_cr' in condition_names:
    plot_interact_effects_modular_roi(f'{rois[0]}', save_dir, 'congruency_switchType_congruencySwitchTypeInteractionSigElectrodes', congruencySwitchTypeInteractionSigElectrodesMeanAndSEM, condition_names, plotting_parameters)

In [ ]:
if len(rois) > 1:
    raise AssertionError("This conflates multiple ROIs right now, just load in one ROI at a time. Sorry, 5/20. Make better later.")
if 'Stimulus_cr' in condition_names:
    plot_interact_effects_modular_roi(f'{rois[0]}', save_dir, 'congruency_switchType_allEffectSensitiveElectrodes', allEffectSensitiveElectrodesAverage, allEffectSensitiveElectrodesSEM, condition_names, plotting_parameters)

plot the four traces for congruency x congruency proportion for electrodes sensitive to the interaction

In [ ]:
if len(rois) > 1:
    raise AssertionError("This conflates multiple ROIs right now, just load in one ROI at a time. Sorry, 5/20. Make better later.")
elif 'Stimulus_c25' in condition_names:
    plot_interact_effects_modular_roi(f'{rois[0]}', save_dir, 'congruency_congruencyProportion_congruencyCongruencyProportionInteractionSigElectrodes', congruencyCongruencyProportionInteractionSigElectrodesMeanAndSEM, condition_names, plotting_parameters)

plot four traces for switch x switch proportion for electrodes sensitive to the interaction

In [ ]:
if len(rois) > 1:
    raise AssertionError("This conflates multiple ROIs right now, just load in one ROI at a time. Sorry, 5/20. Make better later.")
elif 'Stimulus_s25' in condition_names:
    plot_interact_effects_modular_roi(f'{rois[0]}', save_dir, 'switchType_switchProportion_switchTypeSwitchProportionInteractionSigElectrodes', switchTypeSwitchProportionInteractionSigElectrodesMeanAndSEM, condition_names, plotting_parameters)

plot the four traces for switch type x switch proportion for electrodes sensitive to the interaction

plot (ir - cr) vs (is - cs)  
4/5 - functionize this stuff later

SEM diff = sqrt(SEM1^2 + SEM2^2)

In [ ]:
diff_ir_cr = {}  # Difference between IR and CR
diff_is_cs = {}  # Difference between IS and CS

for roi in rois:
    diff_ir_cr[roi] = congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['mean'] - congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['mean']
    diff_is_cs[roi] = congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_is']['mean'] - congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['mean']

sem_diff_ir_cr = {}
sem_diff_is_cs = {}

for roi in rois:
    sem_diff_ir_cr[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['sem'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['sem'], 2))
    sem_diff_is_cs[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_is']['sem'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['sem'], 2))

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, diff_ir_cr[roi], label='IR - CR', color='black', linestyle='-')
ax.plot(times, diff_is_cs[roi], label='IS - CS', color='black', linestyle='--')

ax.fill_between(times, diff_ir_cr[roi] - sem_diff_ir_cr[roi], diff_ir_cr[roi] + sem_diff_ir_cr[roi], alpha=0.2, color='black')
ax.fill_between(times, diff_is_cs[roi] - sem_diff_is_cs[roi], diff_is_cs[roi] + sem_diff_is_cs[roi], alpha=0.2, color='black')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
# ax.set_ylabel('Z-score Difference', fontsize=20)
# ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

plt.tight_layout()
plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_IR-CRvsIS-CS_power.png')
plt.savefig(save_path)
plt.close()

now let's do switch cost as a function of congruency

In [ ]:
diff_is_ir = {}  # Difference between IR and CR
diff_cs_cr = {}  # Difference between IS and CS

# need to change these to the mean and sem format. As shown above.
for roi in rois:
    diff_is_ir[roi] = congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_is']['mean'] - congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['mean']
    diff_cs_cr[roi] = congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['mean'] - congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['mean']

sem_diff_is_ir = {}
sem_diff_cs_cr = {}

for roi in rois:
    sem_diff_is_ir[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_is']['sem'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['sem'], 2))
    sem_diff_cs_cr[roi] = np.sqrt(np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['sem'], 2) + np.power(congruencySwitchTypeInteractionSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['sem'], 2))

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, diff_is_ir[roi], label='IS - IR', color='black', linestyle='-')
ax.plot(times, diff_cs_cr[roi], label='CS - CR', color='black', linestyle='--')

ax.fill_between(times, diff_is_ir[roi] - sem_diff_is_ir[roi], diff_is_ir[roi] + sem_diff_is_ir[roi], alpha=0.2, color='black')
ax.fill_between(times, diff_cs_cr[roi] - sem_diff_cs_cr[roi], diff_cs_cr[roi] + sem_diff_cs_cr[roi], alpha=0.2, color='black')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
ax.set_ylabel('Z-score Difference', fontsize=20)
ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

plt.tight_layout()
plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_IS-IRvsCS-CR_power.png')
plt.savefig(save_path)
plt.close()

let's also plot i vs c 4/5

In [ ]:
avg_ir_is = {}  # Average of IR and IS
avg_cr_cs = {}  # Average of CR and CS

# change to mean and sem formatting as shown above 5/20. But more realistically, make a function to do all of this.
for roi in rois:
    avg_ir_is[roi] = (congruencySigElectrodesMeanAndSEM[roi]['Stimulus_ir']['mean'] + congruencySigElectrodesMeanAndSEM[roi]['Stimulus_is']['mean']) / 2
    avg_cr_cs[roi] = (congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cr']['mean'] + congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cs']['mean']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_is = {}
avg_sem_cr_cs = {}

for roi in rois:
    avg_sem_ir_is[roi] = np.sqrt((np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_ir']['sem'], 2) + np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_is']['sem'], 2)) / 2)
    avg_sem_cr_cs[roi] = np.sqrt((np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cr']['sem'], 2) + np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cs']['sem'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_cr_cs[roi], label='Congruent', color='red', linestyle='-')
ax.plot(times, avg_ir_is[roi], label='Incongruent', color='red', linestyle='--')

ax.fill_between(times, avg_ir_is[roi] - avg_sem_ir_is[roi], avg_ir_is[roi] + avg_sem_ir_is[roi], alpha=0.2, color='red')
ax.fill_between(times, avg_cr_cs[roi] - avg_sem_cr_cs[roi], avg_cr_cs[roi] + avg_sem_cr_cs[roi], alpha=0.2, color='red')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
# ax.set_ylabel('Z-score Difference', fontsize=20)
# ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

plt.tight_layout()
# plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_IvsC_congruencySigElectrodes_power.png')
plt.savefig(save_path)
plt.close()

now lets do i - c for the switch type main effect electrodes

In [ ]:
congruencySigElectrodesMeanAndSEM

In [ ]:
switchTypeSigElectrodesMeanAndSEM

In [ ]:
avg_ir_is = {}  # Average of IR and IS
avg_cr_cs = {}  # Average of CR and CS

for roi in rois:
    avg_ir_is[roi] = (switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['mean'] + switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_is']['mean']) / 2
    avg_cr_cs[roi] = (switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['mean'] + switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['mean']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_is = {}
avg_sem_cr_cs = {}

for roi in rois:
    avg_sem_ir_is[roi] = np.sqrt((np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['sem'], 2) + np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_is']['sem'], 2)) / 2)
    avg_sem_cr_cs[roi] = np.sqrt((np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['sem'], 2) + np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['sem'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_ir_is[roi], label='Congruent', color='red', linestyle='-')
ax.plot(times, avg_cr_cs[roi], label='Incongruent', color='red', linestyle='--')

ax.fill_between(times, avg_ir_is[roi] - avg_sem_ir_is[roi], avg_ir_is[roi] + avg_sem_ir_is[roi], alpha=0.2, color='red')
ax.fill_between(times, avg_cr_cs[roi] - avg_sem_cr_cs[roi], avg_cr_cs[roi] + avg_sem_cr_cs[roi], alpha=0.2, color='red')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
ax.set_ylabel('Z-score Difference', fontsize=20)
ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

plt.tight_layout()
# plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_IvsC_switchTypeSigElectrodes_power.png')
plt.savefig(save_path)
plt.close()

now let's plot switch vs repeat for the switch type main effect electrodes

In [ ]:
avg_ir_cr = {}  # Average of IR and CR
avg_is_cs = {}  # Average of IS and CS

for roi in rois:
    avg_ir_cr[roi] = (switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['mean'] + switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['mean']) / 2
    avg_is_cs[roi] = (switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_is']['mean'] + switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['mean']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_cr = {}
avg_sem_is_cs = {}

for roi in rois:
    avg_sem_ir_cr[roi] = np.sqrt((np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_ir']['sem'], 2) + np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cr']['sem'], 2)) / 2)
    avg_sem_is_cs[roi] = np.sqrt((np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_is']['sem'], 2) + np.power(switchTypeSigElectrodesMeanAndSEM[roi]['Stimulus_cs']['sem'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_ir_cr[roi], label='Repeat', color='blue', linestyle='-')
ax.plot(times, avg_is_cs[roi], label='Switch', color='blue', linestyle='--')

ax.fill_between(times, avg_ir_cr[roi] - avg_sem_ir_cr[roi], avg_ir_cr[roi] + avg_sem_ir_cr[roi], alpha=0.2, color='blue')
ax.fill_between(times, avg_is_cs[roi] - avg_sem_is_cs[roi], avg_is_cs[roi] + avg_sem_is_cs[roi], alpha=0.2, color='blue')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
# ax.set_ylabel('Z-score Difference', fontsize=20)
# ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

plt.tight_layout()
# plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_SvsR_switchTypeSigElectrodes_power.png')
plt.savefig(save_path)
plt.close()

In [ ]:
congruencySigElectrodes

In [ ]:
switchTypeSigElectrodes

In [ ]:
congruencySwitchTypeInteractionSigElectrodes

interestingly, the switch vs repeat for congruency main effect electrodes is quite different too

In [ ]:
avg_ir_cr = {}  # Average of IR and CR
avg_is_cs = {}  # Average of IS and CS

for roi in rois:
    avg_ir_cr[roi] = (congruencySigElectrodesMeanAndSEM[roi]['Stimulus_ir']['mean'] + congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cr']['mean']) / 2
    avg_is_cs[roi] = (congruencySigElectrodesMeanAndSEM[roi]['Stimulus_is']['mean'] + congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cs']['mean']) / 2

# assuming equal sample sizes, which i think we should have
avg_sem_ir_cr = {}
avg_sem_is_cs = {}

for roi in rois:
    avg_sem_ir_cr[roi] = np.sqrt((np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_ir']['sem'], 2) + np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cr']['sem'], 2)) / 2)
    avg_sem_is_cs[roi] = np.sqrt((np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_is']['sem'], 2) + np.power(congruencySigElectrodesMeanAndSEM[roi]['Stimulus_cs']['sem'], 2)) / 2)

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_power_rescaled'].times

roi = 'lpfc'

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, avg_ir_cr[roi], label='repeat', color='blue', linestyle='-')
ax.plot(times, avg_is_cs[roi], label='switch', color='blue', linestyle='--')

ax.fill_between(times, avg_ir_cr[roi] - avg_sem_ir_cr[roi], avg_ir_cr[roi] + avg_sem_ir_cr[roi], alpha=0.2, color='blue')
ax.fill_between(times, avg_is_cs[roi] - avg_sem_is_cs[roi], avg_is_cs[roi] + avg_sem_is_cs[roi], alpha=0.2, color='blue')

# # Overlay a dotted vertical line at time = 0.5
# ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

# Remove top and right borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
ax.set_ylabel('Z-score Difference', fontsize=20)
ax.legend(fontsize=20, loc='upper left')

# Make the x and y ticks bigger
ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

plt.tight_layout()
# plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_SvsR_congruencySigElectrodes_power.png')
plt.savefig(save_path)
plt.close()

### make bar plots windowed from 0 to 1 s of my two conditions 4/9  
greg wants to see these

first get mean and sem across electrodes from 0 to 1 for various conditions

In [ ]:
def calculate_means_sems_from_fullSecond_averages(timeAvg_fullSecond_lists, rois, condition_names):
    means = {}
    sems = {}
    
    for roi in rois:
        means[roi] = {}
        sems[roi] = {}
        for condition_name in condition_names:
            # Handle each electrode's full-second average individually
            full_second_averages = timeAvg_fullSecond_lists[condition_name][roi]
            # Assuming full_second_averages is a list of numpy arrays, one per electrode
            
            # Initialize lists to store the mean for each electrode
            electrode_means = []
            
            for electrode_data in full_second_averages:
                # Calculate the mean for this electrode in the full second window
                electrode_mean = np.nanmean(electrode_data)
                electrode_means.append(electrode_mean)
            
            # Convert the list of means to a NumPy array for further calculation
            electrode_means = np.array(electrode_means)
            
            # Calculate the overall mean and SEM across electrodes
            condition_mean = np.nanmean(electrode_means)
            condition_sem = np.std(electrode_means, ddof=1) / np.sqrt(len(electrode_means))
            
            means[roi][condition_name] = condition_mean
            sems[roi][condition_name] = condition_sem
    
    return means, sems

# Now, calculate the means and SEMs
means_windowed_full_second, sems_windowed_full_second = calculate_means_sems_from_fullSecond_averages(congruencySwitchTypeInteractionSigElectrodes_timeAvg_fullSecondLists, rois, condition_names)
congruency_means_windowed_full_second, congruency_sems_windowed_full_second = calculate_means_sems_from_fullSecond_averages(congruencySigElectrodes_timeAvg_fullSecondLists, rois, condition_names)
switchType_means_windowed_full_second, switchType_sems_windowed_full_second = calculate_means_sems_from_fullSecond_averages(switchTypeSigElectrodes_timeAvg_fullSecondLists, rois, condition_names)

now plot all four conditions as bars on one plot for the interaction effect 4/10

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Corrected condition names based on your dataset
conditions = [
    'Stimulus_cr',  # Congruent Repeat
    'Stimulus_ir',  # Incongruent Repeat
    'Stimulus_cs',  # Congruent Switch
    'Stimulus_is'   # Incongruent Switch
]

# Retrieve the means and SEMs for each condition for plotting
means = [means_windowed_full_second[roi][cond] for cond in conditions]
sems = [sems_windowed_full_second[roi][cond] for cond in conditions]

# Colors for each condition for plotting
group_colors = ['pink', 'red', 'pink', 'red']  # Mapping colors to Congruent/Incongruent

# Plotting
fig, ax = plt.subplots()
bar_width = 0.35  # Width of the bars
index = np.arange(2)  # Two groups: Repeat and Switch

# Creating bars for each group
for i, (mean, sem, color) in enumerate(zip(means, sems, group_colors)):
    position = index[i // 2] + (i % 2 - 0.5) * bar_width
    ax.bar(position, mean, yerr=sem, capsize=5, color=color, width=bar_width)

# Ensure no labels or ticks are shown on the x-axis
ax.set_xticks([])  # No x-tick marks
ax.tick_params(axis='x', which='both', length=0)  # No x-tick marks
ax.tick_params(axis='y', labelsize=32)  # Adjust y-tick label size as needed

# Customizing the plot (commented sections are optional customizations)
# ax.set_ylabel('Average Z-score', fontsize=14)
# ax.set_title('Average Z-score From Baseline by Condition and Type (Full Second)', fontsize=16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
# plt.legend()
save_path = os.path.join(save_dir, f'avg_{roi}_CSvsCRvsIRvsIS_fullSecond.png')
plt.savefig(save_path)
plt.close()


now plot congruent vs incongruent 4/10

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming roi, congruency_means_windowed_full_second, and congruency_sems_windowed_full_second are defined

# Conditions based on your specification
conditions = [
    'Stimulus_cr_fixationCrossBase_1sec_mirror',  # Congruent Repeat
    'Stimulus_ir_fixationCrossBase_1sec_mirror',  # Incongruent Repeat
    'Stimulus_cs_fixationCrossBase_1sec_mirror',  # Congruent Switch
    'Stimulus_is_fixationCrossBase_1sec_mirror'   # Incongruent Switch
]

# Calculating combined means and SEMs for Congruent and Incongruent conditions
combined_means = [
    np.mean([
        congruency_means_windowed_full_second[roi][conditions[0]],  # CR mean
        congruency_means_windowed_full_second[roi][conditions[2]]   # CS mean
    ]),
    np.mean([
        congruency_means_windowed_full_second[roi][conditions[1]],  # IR mean
        congruency_means_windowed_full_second[roi][conditions[3]]   # IS mean
    ])
]

combined_sems = [
    np.sqrt(np.mean([
        congruency_sems_windowed_full_second[roi][conditions[0]]**2,  # CR SEM^2
        congruency_sems_windowed_full_second[roi][conditions[2]]**2   # CS SEM^2
    ])),
    np.sqrt(np.mean([
        congruency_sems_windowed_full_second[roi][conditions[1]]**2,  # IR SEM^2
        congruency_sems_windowed_full_second[roi][conditions[3]]**2   # IS SEM^2
    ]))
]

# Plotting adjustments for the first plot
fig, ax = plt.subplots()
bar_width = 0.015  # Narrower bars

# Adjust the index to create slight separation
index = np.array([0.4375, 0.4625])  # Adjusted for slight separation

bars = ax.bar(index, combined_means, yerr=combined_sems, capsize=5, color=['pink', 'red'], width=bar_width)

# Customizing the plot
ax.set_xticks(index)
ax.set_xticklabels(['Congruent', 'Incongruent'], fontsize=24)
ax.tick_params(axis='y', labelsize=24)

plt.tight_layout()
plt.show()

# Plotting adjustments for the second plot with same formatting
fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size as needed

# Reusing the same bar width and indices for consistency
bars = ax.bar(index, combined_means, 
              yerr=combined_sems, capsize=5, 
              color=['pink', 'red'], width=bar_width)

ax.set_xticks(index)
ax.set_xticklabels(['Congruent', 'Incongruent'], fontsize=32)
ax.tick_params(axis='y', labelsize=32)

# Optional: Uncomment the next line if you want to see the effect without plt.tight_layout()
# plt.tight_layout()

plt.show()


now plot switch vs repeat windowed from 0 to 1

In [ ]:
combined_means_switch_repeat = [
    np.mean([
        switchType_means_windowed_full_second[roi][conditions[0]],  # CR mean
        switchType_means_windowed_full_second[roi][conditions[1]]   # IR mean
    ]),
    np.mean([
        switchType_means_windowed_full_second[roi][conditions[2]],  # CS mean
        switchType_means_windowed_full_second[roi][conditions[3]]   # IS mean
    ])
]

combined_sems_switch_repeat = [
    np.sqrt(np.mean([
        switchType_sems_windowed_full_second[roi][conditions[0]]**2,  # CR SEM^2
        switchType_sems_windowed_full_second[roi][conditions[1]]**2   # IR SEM^2
    ])),
    np.sqrt(np.mean([
        switchType_sems_windowed_full_second[roi][conditions[2]]**2,  # CS SEM^2
        switchType_sems_windowed_full_second[roi][conditions[3]]**2   # IS SEM^2
    ]))
]

fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size as needed
bar_width = 0.015  # Even narrower bars

# Closer indices, but ensure they're distinct enough to not overlap
index_switch_repeat = np.array([0.4375, 0.4625])

bars = ax.bar(index_switch_repeat, combined_means_switch_repeat, 
              yerr=combined_sems_switch_repeat, capsize=5, 
              color=['lightblue', 'blue'], width=bar_width)

ax.set_xticks(index_switch_repeat)
ax.set_xticklabels(['Repeat', 'Switch'], fontsize=24)
ax.tick_params(axis='y', labelsize=24)

# Uncomment the next line if you want to see the effect without plt.tight_layout()
# plt.tight_layout()

plt.show()


### plot individual electrodes for interaction effects
i think this will just work regardless of the output names 3/5

test this new plot significance function that offsets for each significance bar 4/7

In [ ]:
def plot_significance(ax, times, sig_effects, y_offset=0.1):
    """
    Plot significance bars for the effects on top of the existing axes, adjusted for time windows.

    Parameters:
    - ax: The matplotlib Axes object to plot on.
    - times: Array of time points for the x-axis.
    - sig_effects: Dictionary with time windows as keys and lists of tuples (effect, p-value) as values.
    - y_offset: The vertical offset between different time window significance bars.
    """
    y_pos_base = ax.get_ylim()[1]  # Get the top y-axis limit to place significance bars

    time_windows = {
        'firstHalfSecond': (0, 0.5),
        'secondHalfSecond': (0.5, 1),
        'fullSecond': (0, 1)
    }

    window_offsets = {window: 0 for window in time_windows}  # Initialize offsets for each time window

    # Sort time windows to ensure 'FullSecond' bars are plotted last (on top)
    for time_window, effects in sorted(sig_effects.items(), key=lambda x: x[0] == 'fullSecond'):
        base_y_pos = y_pos_base + y_offset * list(time_windows).index(time_window)
        for effect, p_value in effects:
            start_time, end_time = time_windows[time_window]
            # Adjust y_pos based on how many bars have already been plotted in this window
            y_pos = base_y_pos + y_offset * window_offsets[time_window]

            # Update the color selection logic as per your requirement
            color = 'black'  # Default color for unmatched conditions
                        
            if 'congruency' in effect and 'congruencyProportion' not in effect and 'switchType' not in effect and 'congruency:congruencyProportion' not in effect and 'congruency:switchType' not in effect:
                color = 'red'
            elif 'congruencyProportion' in effect and 'switchType' not in effect and 'congruency:congruencyProportion' not in effect and 'congruency:switchType' not in effect:
                color = 'pink'
            elif 'switchType' in effect and 'switchProportion' not in effect and 'congruency' not in effect and 'switchType:switchProportion' not in effect and 'congruency:switchType' not in effect:
                color = 'blue'
            elif 'switchProportion' in effect and 'congruency' not in effect and 'switchType:switchProportion' not in effect and 'congruency:switchType' not in effect:
                color = 'skyblue'
            elif 'congruency:congruencyProportion' in effect:
                color = 'hotpink'
            elif 'switchType:switchProportion' in effect:
                color = 'gray'
            elif 'congruency:switchType' in effect:
                color = 'black'

            num_asterisks = '*' * (1 if p_value < 0.05 else 2 if p_value < 0.01 else 3)
            ax.plot([start_time, end_time], [y_pos, y_pos], color=color, lw=4)
            ax.text((start_time + end_time) / 2, y_pos, num_asterisks, ha='center', va='bottom', color=color)

            window_offsets[time_window] += 1  # Increment the offset for this time window

In [ ]:
LAB_root = None
channels = None

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

    
# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_rescaled'].times

# Define the time windows
time_windows = ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']

# port over the plot_electrodes_grid_whole_brain_analysis here, but replace wherever the save name is wholebrainanalysis with the roi names. 3/25.
def plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters):

    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        for condition_name in condition_names:
            color = plotting_parameters[condition_name]['color']
            line_style = plotting_parameters[condition_name]['line_style']
            ax.plot(times, data[condition_name], label=f'{roi}_{condition_name}', color=color, linestyle=line_style)
            ax.fill_between(times, 
                            data[condition_name] - np.std(data[condition_name], ddof=1) / np.sqrt(len(data[condition_name])),
                            data[condition_name] + np.std(data[condition_name], ddof=1) / np.sqrt(len(data[condition_name])), alpha=0.3)

        # Overlay a dotted vertical line at time = 0.5
        # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

        # Retrieve significant effects for the current subject and electrode
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {}).get(roi, {})
        if sig_effects:
            # Adjust y_offset based on plotting needs. This used to not be assigned to a variable. 3/20.
            plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()


test looping over subjects and electrodes as a function 4/1

THIS NEEDS TO GRAB SPECIFIC ROIS FROM THE SIGNIFICANT EFFECTS STRUCTURE. ALSO NEED TO GRAB THE CORRECT SAVE NAME. 6/11

In [ ]:
def plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, roi, rois_suffix, concatenated_trialAvg_data, condition_names, grid_size, save_dir, save_name, times, plotting_parameters):
    electrodes_data = []
    electrode_counter = 0
    grid_num = 0

    # Load in significant effects structure
    significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_significantEffectsStructure_{rois_suffix}.txt')
    with open(significant_effects_structure_file_path, 'r') as file:
        significant_effects_structure = json.load(file)

    for sub in subjects:
        if sub in sig_electrodes_per_subject_roi[roi]:
            for electrode in sig_electrodes_per_subject_roi[roi][sub]:
                electrode_data = {}
                for condition_name in condition_names:
                    # Ensure the index is correctly used here for your data structure
                    electrode_data[condition_name] = concatenated_trialAvg_data[condition_name][roi][electrode_counter]

                electrodes_data.append((electrode_data, sub, electrode))
                electrode_counter += 1
                if len(electrodes_data) == grid_size:
                    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
                    electrodes_data = []  # Reset for the next grid
                    grid_num += 1

    # Plot remaining electrodes in the last grid
    if electrodes_data:
        plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)


loop through rois and plot individual electrodes with color-coded significance bars

In [ ]:
for roi in rois:
    plot_electrodes_grid_roi_loop(subjects, sig_electrodes_per_subject_roi, roi, rois_suffix, concatenated_trialAvg_data, condition_names, 16, save_dir, conditions_save_name, times, plotting_parameters)

awful godforsaken code to get individual electrode plots for congruency main effects, switch main effects, and interaction effects for lpfc 4/8  
replace with function later after CNS...    

In [ ]:
LAB_root = None
channels = None

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_condition_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_condition_name]['HG_ev1_evoke_rescaled'].times

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json

def plot_electrodes_grid_roi_switchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate S and R for the electrode
        avg_ir_cr = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_cr_fixationCrossBase_1sec_mirror']) / 2
        avg_is_cs = (data['Stimulus_is_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
        # avg_sem_is_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2)) / 2)
        # avg_sem_cr_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

        # Plotting S and R
        ax.plot(times, avg_ir_cr, label='repeat', color='blue', linestyle='-')
        ax.plot(times, avg_is_cs, label='switch', color='blue', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            utils.plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()


def plot_electrodes_grid_roi_congruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate I-C for the electrode
        avg_ir_is = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_is_fixationCrossBase_1sec_mirror']) / 2
        avg_cr_cs = (data['Stimulus_cr_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
        # avg_sem_is_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_ir_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cr_fixationCrossBase_1sec_mirror'], 2)) / 2)
        # avg_sem_cr_cs = np.sqrt((np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_is_fixationCrossBase_1sec_mirror'], 2) + np.power(switchTypeSigElectrodesSEM[roi]['Stimulus_cs_fixationCrossBase_1sec_mirror'], 2)) / 2)

        # Plotting I-C difference
        ax.plot(times, avg_cr_cs, label='congruent', color='red', linestyle='-')
        ax.plot(times, avg_ir_is, label='incongruent', color='red', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            utils.plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()



def plot_electrodes_grid_roi_congruencyEffectSwitchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate congruency effect as a function of switch type for the electrode

        avg_diff_ir_cr = data['Stimulus_ir_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']
        avg_diff_is_cs = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_cs_fixationCrossBase_1sec_mirror']


        ax.plot(times, avg_diff_ir_cr, label='IR - CR', color='black', linestyle='-')
        ax.plot(times, avg_diff_is_cs, label='IS - CS', color='black', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1) #change back to plot_significance after poster 4/8

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()



def plot_electrodes_grid_roi_switchCostCongruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        # Calculate congruency effect as a function of switch type for the electrode

        avg_diff_is_ir = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_ir_fixationCrossBase_1sec_mirror']
        avg_diff_cs_cr = data['Stimulus_cs_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']


        ax.plot(times, avg_diff_is_ir, label='IS - IR', color='black', linestyle='-')
        ax.plot(times, avg_diff_cs_cr, label='CS - CR', color='black', linestyle='--')
        
        # Overlay a dotted vertical line at time = 0.5
        ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

        # Remove top and right borders
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time from Stimulus Onset (s)')
        ax.set_ylabel('Z-score From Baseline')

        # Retrieve and plot significant effects
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1) #change back to plot_significance after poster 4/8

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()



try this for lpfc 4/8  
just for poster, clean this up!! this is hella hard-coded 

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'congruencySigElectrodesCongruencyComparison'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access congruencySigElectrodes for sub
    # If sub is not a key, congruencySigElectrodes_for_sub will be None
    congruencySigElectrodes_for_sub = congruencySigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if congruencySigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in congruencySigElectrodes_for_sub:
        electrode_data = {}
        for condition_name in condition_names:
            electrode_data[condition_name] = concatenated_trialAvg_data_congruencySigElectrodes[roi][condition_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_congruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_congruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

okay now do individual for switch type

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'switchTypeSigElectrodesSwitchTypeComparison'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access switchType for sub
    # If sub is not a key, switchTypeSigElectrodes_for_sub will be None
    switchTypeSigElectrodes_for_sub = switchTypeSigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if switchTypeSigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in switchTypeSigElectrodes_for_sub:
        electrode_data = {}
        for condition_name in condition_names:
            electrode_data[condition_name] = concatenated_trialAvg_data_switchTypeSigElectrodes[roi][condition_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_switchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_switchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

now do individual for interaction (congruency effect by switch type)

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'congruencySwitchTypeInteractionSigElectrodesSwitchTypeComparison'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access switchType for sub
    # If sub is not a key, switchTypeSigElectrodes_for_sub will be None
    congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if congruencySwitchTypeInteractionSigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in congruencySwitchTypeInteractionSigElectrodes_for_sub:
        electrode_data = {}
        for condition_name in condition_names:
            electrode_data[condition_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][condition_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_congruencyEffectSwitchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_congruencyEffectSwitchTypeWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

now do individual for interaction (switch cost by congruency)

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
save_name = 'switchCostCongruencyInteractionSigElectrodes'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
for sub in subjects:

    # Use .get() to safely access switchType for sub
    # If sub is not a key, switchTypeSigElectrodes_for_sub will be None
    congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

    # Check if congruencySigElectrodes_for_sub is None (i.e., if sub was not a key in congruencySigElectrodes)
    if congruencySwitchTypeInteractionSigElectrodes_for_sub is None:
        continue  # Skip this sub and move to the next one

    # If we reach here, it means congruencySigElectrodes_for_sub is not None, and we can safely use it
    for electrode in congruencySwitchTypeInteractionSigElectrodes_for_sub:
        electrode_data = {}
        for condition_name in condition_names:
            electrode_data[condition_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][condition_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:
            plot_electrodes_grid_roi_switchCostCongruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi_switchCostCongruencyWithInteractionOutputNames(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

okay once we choose the electrodes we want, plot the single example electrode

for congruency

In [ ]:
def plot_single_electrode_data_congruency(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size
    avg_ir_is = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_is_fixationCrossBase_1sec_mirror']) / 2
    avg_cr_cs = (data['Stimulus_cr_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
    
    ax.plot(times, avg_cr_cs, label='Congruent', color='red', linestyle='-')
    ax.plot(times, avg_ir_is, label='Incongruent', color='red', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    plt.tight_layout()

    # Incorporate significance plotting
    if sig_effects:
        utils.plot_significance(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0063', 'RMMF13'
roi = 'lpfc'
electrode_index = 5 # right now just manually count this from the grid plot BUT make this real after CNS 4/9

save_name = 'congruencySigElectrodesCongruencyComparison'
congruencySigElectrodes_for_sub = congruencySigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for condition_name in condition_names:
    electrode_data[condition_name] = concatenated_trialAvg_data_congruencySigElectrodes[roi][condition_name][electrode_index]

plot_single_electrode_data_congruency(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

for switchType

In [ ]:
def plot_single_electrode_data_switchType(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size
    avg_ir_cr = (data['Stimulus_ir_fixationCrossBase_1sec_mirror'] + data['Stimulus_cr_fixationCrossBase_1sec_mirror']) / 2
    avg_is_cs = (data['Stimulus_is_fixationCrossBase_1sec_mirror'] + data['Stimulus_cs_fixationCrossBase_1sec_mirror']) / 2
    
    ax.plot(times, avg_ir_cr, label='Repeat', color='blue', linestyle='-')
    ax.plot(times, avg_is_cs, label='Switch', color='blue', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)

    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    plt.tight_layout()

    # Incorporate significance plotting
    if sig_effects:
        plot_significance(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0059', 'LMMF9'
roi = 'lpfc'
electrode_index = 1 # right now just manually get this from the grid plot BUT make this real after CNS 4/9

save_name = 'switchTypeSigElectrodesCongruencyComparison'
switchTypeSigElectrodes_for_sub = switchTypeSigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for condition_name in condition_names:
    electrode_data[condition_name] = concatenated_trialAvg_data_switchTypeSigElectrodes[roi][condition_name][electrode_index]

plot_single_electrode_data_switchType(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

for interaction effect, congruency effect by switch type

In [ ]:
def plot_single_electrode_data_interaction_effect(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size

    avg_diff_ir_cr = data['Stimulus_ir_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']
    avg_diff_is_cs = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_cs_fixationCrossBase_1sec_mirror']

    ax.plot(times, avg_diff_ir_cr, label='IR - CR', color='black', linestyle='-')
    ax.plot(times, avg_diff_is_cs, label='IS - CS', color='black', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)
    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    plt.tight_layout()

    # Incorporate significance plotting
    if sig_effects:
        plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0094', 'LFAI9'
electrode_index = 8 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'congruencySwitchTypeInteractionSigElectrodesSwitchTypeComparison'
congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for condition_name in condition_names:
    electrode_data[condition_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][condition_name][electrode_index]
plot_single_electrode_data_interaction_effect(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

In [ ]:
congruencySwitchTypeInteractionSigElectrodes

interaction effect for switch cost by congruency

In [ ]:
def plot_single_electrode_data_interaction_effect(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size

    avg_diff_is_ir = data['Stimulus_is_fixationCrossBase_1sec_mirror'] - data['Stimulus_ir_fixationCrossBase_1sec_mirror']
    avg_diff_cs_cr = data['Stimulus_cs_fixationCrossBase_1sec_mirror'] - data['Stimulus_cr_fixationCrossBase_1sec_mirror']

    ax.plot(times, avg_diff_is_ir, label='IS - IR', color='black', linestyle='-')
    ax.plot(times, avg_diff_cs_cr, label='CS - CR', color='black', linestyle='--')

    # ax.axvline(x=0.5, color='k', linestyle='--', linewidth=1)
    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    ax.set_ylabel('Z-score Difference', fontsize=20)
    ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=20)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=20)  # Adjust y-axis tick label size

    # Incorporate significance plotting
    if sig_effects:
        plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0094', 'LFAI9'
electrode_index = 8 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'congruencySwitchTypeInteractionSigElectrodesSwitchTypeComparison'
congruencySwitchTypeInteractionSigElectrodes_for_sub = congruencySwitchTypeInteractionSigElectrodes.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for condition_name in condition_names:
    electrode_data[condition_name] = concatenated_trialAvg_data_congruencySwitchTypeInteractionSigElectrodes[roi][condition_name][electrode_index]
plot_single_electrode_data_interaction_effect(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

plot example electrode with all four conditions  
note this index is in the overall sig electrodes

In [ ]:
def plot_single_electrode_data_interaction_effect_all_four_conditions(data, times, electrode, subject_id, sig_effects, save_dir, save_name):
    fig, ax = plt.subplots(figsize=(10, 6))  # A more focused figure size

    avg_is = data['Stimulus_is_fixationCrossBase_1sec_mirror']
    avg_ir = data['Stimulus_ir_fixationCrossBase_1sec_mirror']
    avg_cs = data['Stimulus_cs_fixationCrossBase_1sec_mirror']
    avg_cr = data['Stimulus_cr_fixationCrossBase_1sec_mirror']

    ax.plot(times, avg_is, label='IS', color='red', linestyle='--')
    ax.plot(times, avg_ir, label='IR', color='red', linestyle='-')
    ax.plot(times, avg_cs, label='CS', color='pink', linestyle='--')
    ax.plot(times, avg_cr, label='CR', color='pink', linestyle='-')

    # Remove top and right borders
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # ax.set_xlabel('Time from Stimulus Onset (s)', fontsize=20)
    # ax.set_ylabel('Z-score Difference', fontsize=20)
    # ax.legend(fontsize=20, loc='upper left')

    # Make the x and y ticks bigger
    ax.tick_params(axis='x', labelsize=24)  # Adjust x-axis tick label size
    ax.tick_params(axis='y', labelsize=24)  # Adjust y-axis tick label size

    # Incorporate significance plotting
    if sig_effects:
        plot_significance_justInteraction_delete_after_poster(ax, times, sig_effects, y_offset=0.1)

    plt.tight_layout()
    plt.show()

    if save_dir and save_name:
        fig.savefig(os.path.join(save_dir, f'{save_name}_{subject_id}_{electrode}.png'))
    plt.close(fig)

# Example usage setup
sub, example_elec = 'D0094', 'LPAI9'
electrode_index = 31 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'sigElectrodesPerSubjectROI_D0094LPAI9'
sig_electrodes_for_sub = sig_electrodes_per_subject_roi.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for condition_name in condition_names:
    electrode_data[condition_name] = concatenated_trialAvg_data[roi][condition_name][electrode_index]
plot_single_electrode_data_interaction_effect_all_four_conditions(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

In [ ]:
# Example usage setup
sub, example_elec = 'D0065', 'RASF14'
electrode_index = 14 # right now just manually get this from the grid plot BUT make this real after CNS 4/9
roi = 'lpfc'
save_name = 'sigElectrodesPerSubjectROI_D0065_RASF14'
sig_electrodes_for_sub = sig_electrodes_per_subject_roi.get(sub)

# Load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# Extract significant effects for the specific electrode and subject
sig_effects = significant_effects_structure.get(sub, {}).get(example_elec, {})

electrode_data = {}
for condition_name in condition_names:
    electrode_data[condition_name] = concatenated_trialAvg_data[roi][condition_name][electrode_index]
plot_single_electrode_data_interaction_effect_all_four_conditions(electrode_data, times, example_elec, sub, sig_effects, save_dir, save_name)

old way of looping without a function 4/1

dlpfc

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'dlpfc'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for condition_name in condition_names:
                electrode_data[condition_name] = concatenated_trialAvg_data[roi][condition_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

acc

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'acc'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for condition_name in condition_names:
                electrode_data[condition_name] = concatenated_trialAvg_data[roi][condition_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

parietal

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'parietal'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for condition_name in condition_names:
                electrode_data[condition_name] = concatenated_trialAvg_data[roi][condition_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)

lpfc

In [ ]:
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'lpfc'
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in condition_names:
    save_name = 'congruency_switchType'

# load in significant effects structure
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)

for sub in subjects:
    if sub in sig_electrodes_per_subject_roi[roi]:
        for electrode in sig_electrodes_per_subject_roi[roi][sub]:
            
            electrode_data = {}
            for condition_name in condition_names:
                electrode_data[condition_name] = concatenated_trialAvg_data[roi][condition_name][electrode_counter]

            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1
            if len(electrodes_data) == grid_size:
                plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_roi(electrodes_data, significant_effects_structure, grid_num, roi, condition_names, times, save_dir, save_name, plotting_parameters)